# RioBase - Script voor Data Preparatie

- Datum huidige versie: 2022-10-19
- Auteur: William Hazel

Binnen RioBase is ervoor gekozen om de inspectiebeelden van leidingen te gebruiken om een risicomodel te maken die aangeeft welke leidingen het belangrijkst zijn/het meest aan vervanging toe zijn. Er is veel data preparatie nodig om de codes van de inspectiebeelden bruikbaar te maken. De script is ook vrij lang met veel verschillende stappen.

Er zijn ook twee functies om data te selecteren die gebruikt kunnen worden voor machine learning.

# Libraries Importeren
Meerdere zogenaamde libraries zijn nodig om de data in te kunnen lezen en te verwerken. ElementTree (ET) wordt gebruikt om de .ribx (XML structuur) bestanden in te lezen. Shapely wordt gebruikt om de .shp bestanden van de leidingen en knooppunten in te lezen.

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import re
import math

import matplotlib.pyplot as plt

import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup

from shapely.geometry import Point, Polygon, MultiPolygon
from shapely import wkt
import shapely.wkt                                             # From Shapely
import shapefile                                               # From Pyshp
from pyproj import Proj, transform, CRS, Transformer

# Data Importeren - Classificatie Tabellen
Er is een aantal tabellen nodig om de inspecties te kunnen classificeren. Deze zijn vooraf al bepaald. De parameters en waarden in deze tabellen bepalen dus meteen ook de eindresultaten van deze script.

In [2]:
os.chdir(r'C:\Users\haze0029\OneDrive - HZ University of Applied Sciences\Documents\Projects\Project_DataScience\RIOBASE\RioBaseOplevering\Datavelden_Units_Proxies')



list_cols_maatstaf = ['col1',
                      'Hoofdcode','Karakterisering 1 code','Toestandsaspect','Waa_ondg','Waa_bovg','Ing_ondg','Ing_bovg',
                      'Faalwijze','faalkans_klasse1_basis','faalkans_klasse2_1binnenxm','faalkans_klasse2_meerdere',
                      'faalkans_klasse3_1binnenxm','faalkans_klasse3_meerdere',
                      'faalkans_klasse4_1binnenxm','faalkans_klasse4_meerdere',
                      'faalkans_klasse5_1binnenxm','faalkans_klasse5_meerdere','col2']

list_cols_tabel_BAJA = ['col1','mm_breedte','mm_hoogte','mm_diameter',
                       'beton_mof_k1_gd','beton_mof_k1_koga','beton_mof_k2_gd','beton_mof_k2_koga',
                       'beton_mof_k3_gd','beton_mof_k3_koga','beton_mof_k4_gd','beton_mof_k4_koga',
                       'beton_mof_k5_gd','beton_mof_k5_koga',
                       'beton_vaar_k1_gd','beton_vaar_k1_koga','beton_vaar_k2_gd','beton_vaar_k2_koga',
                       'beton_vaar_k3_gd','beton_vaar_k3_koga','beton_vaar_k4_gd','beton_vaar_k4_koga',
                       'beton_vaar_k5_gd','beton_vaar_k5_koga',
                       'kunstof_mof_k1_gd','kunstof_mof_k1_koga','kunstof_mof_k2_gd','kunstof_mof_k2_koga',
                       'kunstof_mof_k3_gd','kunstof_mof_k3_koga','kunstof_mof_k4_gd','kunstof_mof_k4_koga',
                       'kunstof_mof_k5_gd','kunstof_mof_k5_koga',
                       'overig_k1_gd','overig_k1_koga','overig_k2_gd','overig_k3_koga',
                       'overig_k3_gd','overig_k3_koga','overig_k4_gd','overig_k4_koga',
                       'overig_k5_gd','overig_k5_koga']

# Zitten rijen met "Niet Meegenomen", deze rijen negeren! d.m.v. nrows
df_maatstaf_raw = pd.read_excel('Faalkans_Inputbestand.xlsx',
                                sheet_name=0,names=list_cols_maatstaf,nrows=26,skiprows=2)

df_classificatie_leiding_raw = pd.read_excel('Faalkans_Inputbestand.xlsx',
                                             sheet_name=3,header=0)

df_tabel_BAJA_raw = pd.read_excel('Faalkans_Inputbestand.xlsx',
                                  sheet_name=5,names=list_cols_tabel_BAJA,nrows=27,skiprows=6)


### Functie - Totale Code Maken
Een inspectiecode bestaat uit een hoofdcode en kan 1 of 2 extra letters hebben om een klasse verder te specifieren. Er wordt een nieuwe kolom 'code_hckar1kar2' aangemaakt.

In [3]:
def func_code_hckar1kar2(hoofdcode,kar1,kar2):
    
    new_code_temp = hoofdcode + kar1 + kar2
    new_code = new_code_temp.replace('#','')
    return new_code

df_classificatie_leiding = df_classificatie_leiding_raw.copy()
df_classificatie_leiding.rename(columns=lambda x: x.strip(), inplace=True)

df_classificatie_leiding['code_hckar1kar2'] = df_classificatie_leiding.apply(lambda row: 
        func_code_hckar1kar2(row['Hoofdcode'],row['Karakterisering 1 code'],row['Karakterisering 2 code']),axis=1)

df_tabel_BAJA = df_tabel_BAJA_raw.drop(labels=['col1'],axis=1)

df_maatstaf = df_maatstaf_raw[['Hoofdcode','Karakterisering 1 code','Toestandsaspect',
                                'Waa_ondg','Waa_bovg','Ing_ondg','Ing_bovg']].copy()

df_faalkans = df_maatstaf_raw[['Hoofdcode','Karakterisering 1 code','Toestandsaspect',
                               'Faalwijze','faalkans_klasse1_basis',
                               'faalkans_klasse2_1binnenxm','faalkans_klasse2_meerdere',
                               'faalkans_klasse3_1binnenxm','faalkans_klasse3_meerdere',
                               'faalkans_klasse4_1binnenxm','faalkans_klasse4_meerdere',
                               'faalkans_klasse5_1binnenxm','faalkans_klasse5_meerdere']].copy()


### Dictionaries voor BAJA
Dictionary is nodig om materiaalklassen te generaliseren naar beton, kunstof of overig. Lists zijn nodig om delen van een tabel te selecteren.

In [4]:

dict_BAJA_mat = {'Asbestcement': 'Overig',
                'Bitumen': 'Overig',
                'Bitumen-houtvezel composiet': 'Overig',
                'Metselwerk (baksteen)': 'Overig',
                'Klei': 'Overig',
                'Cementmortel': 'Beton',
                'Beton': 'Beton',
                'Beton-Relined': 'Beton',
                'Gewapend beton': 'Beton',
                'Gespoten beton': 'Beton',
                'Betonnen segmenten': 'Beton',
                'Vezelcement': 'Beton',
                'Epoxy': 'Kunstof',
                'Glasvezel versterkte kunststof': 'Kunstof',
                'GVK': 'Kunstof',
                'Gietijzer': 'Overig',
                'Gres': 'Overig',
                'Grijs gietijzer': 'Overig',
                'Nodulair gietijzer': 'Overig',
                'Staal': 'Overig',
                'Metselwerk (bepleisterd)': 'Overig',
                'Metselwerk (onbepleisterd)': 'Overig',
                'Epoxy': 'Kunstof',
                'Polyester': 'Kunstof',
                'Polyetheen': 'Kunstof',
                'PE': 'Kunstof',
                'Polypropyleen': 'Kunstof',
                'Polyvinylchloride': 'Kunstof',
                'Pvc': 'Kunstof',
                'Unidentified type of plastics': 'Kunstof',
                'Unidentified material': 'Overig',
                'Ultrarib': 'Overig',
                'Anders': 'Overig',
                'Onbekend': 'Overig',
                'nan': '',
                '': ''}

dict_BAJA_klasse = {'beton_mof_k1_gd': 1,
                    'beton_mof_k1_koga': 1,
                    'beton_mof_k2_gd': 2,
                    'beton_mof_k2_koga': 2,
                    'beton_mof_k3_gd': 3,
                    'beton_mof_k3_koga': 3,
                    'beton_mof_k4_gd': 4,
                    'beton_mof_k4_koga': 4,
                    'beton_mof_k5_gd': 5,
                    'beton_mof_k5_koga': 5,
                    'beton_vaar_k1_gd': 1,
                    'beton_vaar_k1_koga': 1,
                    'beton_vaar_k2_gd': 2,
                    'beton_vaar_k2_koga': 2,
                    'beton_vaar_k3_gd': 3,
                    'beton_vaar_k3_koga': 3,
                    'beton_vaar_k4_gd': 4,
                    'beton_vaar_k4_koga': 4,
                    'beton_vaar_k5_gd': 5,
                    'beton_vaar_k5_koga': 5,
                    'kunstof_mof_k1_gd': 1,
                    'kunstof_mof_k1_koga': 1,
                    'kunstof_mof_k2_gd': 2,
                    'kunstof_mof_k2_koga': 2,
                    'kunstof_mof_k3_gd': 3,
                    'kunstof_mof_k3_koga': 3,
                    'kunstof_mof_k4_gd': 4,
                    'kunstof_mof_k4_koga': 4,
                    'kunstof_mof_k5_gd': 5,
                    'kunstof_mof_k5_koga': 5,
                    'overig_k1_gd': 1,
                    'overig_k1_koga': 1,
                    'overig_k2_gd': 2,
                    'overig_k2_koga': 2,
                    'overig_k3_gd': 3,
                    'overig_k3_koga': 3,
                    'overig_k4_gd': 4,
                    'overig_k4_koga': 4,
                    'overig_k5_gd': 5,
                    'overig_k5_koga': 5}

dict_BAJA_gd_koga = {'beton_mof_k1_gd': 'gd',
                     'beton_mof_k1_koga': 'koga',
                     'beton_mof_k2_gd': 'gd',
                     'beton_mof_k2_koga': 'koga',
                     'beton_mof_k3_gd': 'gd',
                     'beton_mof_k3_koga': 'koga',
                     'beton_mof_k4_gd': 'gd',
                     'beton_mof_k4_koga': 'koga',
                     'beton_mof_k5_gd': 'gd',
                     'beton_mof_k5_koga': 'koga',
                     'beton_vaar_k1_gd': 'gd',
                     'beton_vaar_k1_koga': 'koga',
                     'beton_vaar_k2_gd': 'gd',
                     'beton_vaar_k2_koga': 'koga',
                     'beton_vaar_k3_gd': 'gd',
                     'beton_vaar_k3_koga': 'koga',
                     'beton_vaar_k4_gd': 'gd',
                     'beton_vaar_k4_koga': 'koga',
                     'beton_vaar_k5_gd': 'gd',
                     'beton_vaar_k5_koga': 'koga',
                     'kunstof_mof_k1_gd': 'gd',
                     'kunstof_mof_k1_koga': 'koga',
                     'kunstof_mof_k2_gd': 'gd',
                     'kunstof_mof_k2_koga': 'koga',
                     'kunstof_mof_k3_gd': 'gd',
                     'kunstof_mof_k3_koga': 'koga',
                     'kunstof_mof_k4_gd': 'gd',
                     'kunstof_mof_k4_koga': 'koga',
                     'kunstof_mof_k5_gd': 'gd',
                     'kunstof_mof_k5_koga': 'koga',
                     'overig_k1_gd': 'gd',
                     'overig_k1_koga': 'koga',
                     'overig_k2_gd': 'gd',
                     'overig_k3_koga': 'koga',
                     'overig_k3_gd': 'gd',
                     'overig_k3_koga': 'koga',
                     'overig_k4_gd': 'gd',
                     'overig_k4_koga': 'koga',
                     'overig_k5_gd': 'gd',
                     'overig_k5_koga': 'koga'}


list_baja_beton_mof = ['beton_mof_k1_gd','beton_mof_k1_koga','beton_mof_k2_gd','beton_mof_k2_koga',
                       'beton_mof_k3_gd','beton_mof_k3_koga','beton_mof_k4_gd','beton_mof_k4_koga',
                       'beton_mof_k5_gd','beton_mof_k5_koga']

list_baja_beton_vaar = ['beton_vaar_k1_gd','beton_vaar_k1_koga','beton_vaar_k2_gd','beton_vaar_k2_koga',
                        'beton_vaar_k3_gd','beton_vaar_k3_koga','beton_vaar_k4_gd','beton_vaar_k4_koga',
                        'beton_vaar_k5_gd','beton_vaar_k5_koga']

list_baja_kunstof = ['kunstof_mof_k1_gd','kunstof_mof_k1_koga','kunstof_mof_k2_gd','kunstof_mof_k2_koga',
                     'kunstof_mof_k3_gd','kunstof_mof_k3_koga','kunstof_mof_k4_gd','kunstof_mof_k4_koga',
                     'kunstof_mof_k5_gd','kunstof_mof_k5_koga']

list_baja_overig = ['overig_k1_gd','overig_k1_koga','overig_k2_gd','overig_k3_koga',
                    'overig_k3_gd','overig_k3_koga','overig_k4_gd','overig_k4_koga',
                    'overig_k5_gd','overig_k5_koga']

list_pijp_breedtes = [100,200,250,300,315,400,500,600,630,700,800,900,1000,1250,1500,9999]

# Data Importeren - Datavelden
De datavelden (kolomnamen) van de bestanden verschillen per gemeente. Veere en Reimerswaal hanteren dezelfde namen, maar Sluis en Vlissingen doen het op hun eigen manier. Er is een algemene standaard bepaald, maar deze moet toegepast worden op de gemeenten. Door middel van een dictionary worden de kolommen herbenoemd.

In [5]:
os.chdir(r'C:\Users\haze0029\OneDrive - HZ University of Applied Sciences\Documents\Projects\Project_DataScience\RIOBASE\RioBaseOplevering\Datavelden_Units_Proxies')

df_datavelden_lei = pd.read_excel('Datavelden_Lei.xlsx')
df_datavelden_knp = pd.read_excel('Datavelden_Knp.xlsx')
df_datavelden_weg = pd.read_excel('Datavelden_Weg.xlsx')
df_datavelden_units = pd.read_excel('Datavelden_Units.xlsx')

### Kolommen Standaardiseren
In het project Riobase waren er aanvankelijk 4 casus gemeenten, waar Sluis en Vlissingen andere kolomnamen hadden dan Veere en Reimerswaal. De code in de cell hieronder standaardiseert de kolomnamen door middel van dictionaries en .map(). Als er data, lees .shp bestanden, van Open Data Portaal Zeeland wordt gehaald, dan zijn de kolomnamen weer anders. Wel zijn ze gestandaardiseerd. 

In [6]:
list_kolommen_lei = df_datavelden_lei['Dataveld'].to_list()
list_kolommen_knp = df_datavelden_knp['Dataveld'].to_list()
list_kolommen_weg = df_datavelden_weg['Dataveld'].to_list()

# 3 extra nieuwe kolommen toevoegen
list_kolommen_lei.append('coordinates_type')
list_kolommen_lei.append('coordinates_rd')
list_kolommen_lei.append('Gemeente')

# Leidingen
df_kolommen_lei_sluis = df_datavelden_lei[['Dataveld','Sluis']].copy()                # Van gemeente Sluis
df_kolommen_lei_vlissingen = df_datavelden_lei[['Dataveld','Vlissingen']].copy()      # Van gemeente Vlissingen
df_kolommen_lei_portaal = df_datavelden_lei[['Dataveld','Portaal']].copy()            # Van Open Data Portaal Zeeland

dict_rename_lei = {'Sluis': 'Kolom_oud',
                   'Vlissingen': 'Kolom_oud',
                   'Portaal': 'Kolom_oud'}

df_kolommen_lei_sluis.rename(dict_rename_lei,axis=1,inplace=True)
df_kolommen_lei_vlissingen.rename(dict_rename_lei,axis=1,inplace=True)
df_kolommen_lei_portaal.rename(dict_rename_lei,axis=1,inplace=True)
df_kolommen_lei_oud = pd.concat([df_kolommen_lei_sluis,
                                 df_kolommen_lei_vlissingen,
                                 df_kolommen_lei_portaal]).reset_index(drop=True)

dict_rename_kolommen_lei = dict(zip(df_kolommen_lei_oud['Kolom_oud'][df_kolommen_lei_oud['Kolom_oud'] != -1],
                                    df_kolommen_lei_oud['Dataveld'][df_kolommen_lei_oud['Kolom_oud'] != -1]))


# Wegen
df_kolommen_weg_sluis = df_datavelden_weg[['Dataveld','Sluis']].copy()
df_kolommen_weg_veere = df_datavelden_weg[['Dataveld','Veere']].copy()
df_kolommen_weg_vlissingen = df_datavelden_weg[['Dataveld','Vlissingen']].copy()

dict_rename_weg = {'Sluis': 'Kolom_oud',
                   'Veere': 'Kolom_oud',
                   'Vlissingen': 'Kolom_oud'}

df_kolommen_weg_sluis.rename(dict_rename_weg,axis=1,inplace=True)
df_kolommen_weg_veere.rename(dict_rename_weg,axis=1,inplace=True)
df_kolommen_weg_vlissingen.rename(dict_rename_weg,axis=1,inplace=True)
df_kolommen_weg_oud = pd.concat([df_kolommen_weg_sluis,
                                 df_kolommen_weg_veere,
                                 df_kolommen_weg_vlissingen]).reset_index(drop=True)

dict_rename_kolommen_weg = dict(zip(df_kolommen_weg_oud['Kolom_oud'][df_kolommen_weg_oud['Kolom_oud'] != -1],
                                    df_kolommen_weg_oud['Dataveld'][df_kolommen_weg_oud['Kolom_oud'] != -1]))



In [7]:
print(dict_rename_kolommen_lei)

{'WKT': 'wkt_geom', 'LEIDINGNAA': 'LeiNam', 'LEIDINGTYP': 'TypLei', 'BEGINKNOOP': 'KnpNamB', 'EINDKNOOPP': 'KnpNamE', 'BEGINKN_3': 'BobB', 'EINDKNOOP_': 'BobE', 'STRENGLENG': 'LenLei', 'BREEDTE': 'Br', 'HOOGTE': 'Hgt', 'VORM': 'Vrm', 'MATERIAAL': 'Mat', 'FUNDERING': 'TypFun', 'BEGINDATUM': 'BegDat', 'DATUM_AANP': 'DatAanpGep', 'AANPASS_1': 'AnpMth', 'GRONDSLAG_': 'GslAnp', 'INSPECTIED': 'DatInsp', 'WATERDICHT': 'WatDicht', 'STABILITEI': 'Stab', 'AFSTROMING': 'Afs', 'ONDERZOEKS': 'OndzMth', 'WOONPLAATS': 'StrGeb', 'WIJK': 'WijkNam', 'STRAAT': 'StrNam', 'GRONDSOORT': 'Grs', 'GEOVISIA_I': 'BHOBJDC000', 'OBJECTNUMM': 'LeiNam', 'TYPE_DETAI': 'TypLei', 'BEGINPUTNU': 'KnpNamB', 'EINDPUTNUM': 'KnpNamE', 'BOBBEGIN': 'BobB', 'BOBEIND': 'BobE', 'LENGTE': 'LenLei', 'DIAMETER': 'Diam', 'TYPEFUNDER': 'TypFun', 'JAARVANAAN': 'BegDat', 'JAARGEPLAN': 'DatAanpGep', 'JAARLAATST': 'DatInsp', 'OPENBARERU': 'StrNam', 'GEOMETRIE': 'wkt_geom', 'NAAM': 'LeiNam', 'TYPE': 'TypLei', 'BEGINPUNT': 'KnpNamB', 'EINDP

In [8]:
df_datavelden_lei

,Prio,Dataveld,RMW,Sluis,Veere,Vlissingen,Hulst,Portaal
0,1,wkt_geom,wkt_geom,WKT,wkt_geom,WKT,wkt_geom,GEOMETRIE
1,1,BHOBJDC000,BHOBJDC000,-1,BHOBJDC000,GEOVISIA_I,BHOBJDC000,-1
2,1,LeiNam,LeiNam,LEIDINGNAA,LeiNam,OBJECTNUMM,LeiNam,NAAM
3,1,TypLei,TypLei,LEIDINGTYP,TypLei,TYPE_DETAI,TypLei,TYPE
4,1,KnpNamB,KnpNamB,BEGINKNOOP,KnpNamB,BEGINPUTNU,KnpNamB,BEGINPUNT
5,1,KnpNamE,KnpNamE,EINDKNOOPP,KnpNamE,EINDPUTNUM,KnpNamE,EINDPUNT
6,1,BobB,BobB,BEGINKN_3,BobB,BOBBEGIN,BobB,BOBBEGINPU
7,1,BobE,BobE,EINDKNOOP_,BobE,BOBEIND,BobE,BOBEINDPUN
8,1,LenLei,LenLei,STRENGLENG,LenLei,LENGTE,LenLei,LENGTELEID
9,1,Diam,-1,-1,Diam,DIAMETER,-1,DIAMETERLE


In [9]:
df_datavelden_weg

,Prio,Dataveld,RMW,Sluis,Veere,Vlissingen,Hulst,Borsele
0,1,wkt_geom,wkt_geom,WKT,wkt_geom,WKT,wkt_geom,wkt_geom
1,1,WVL_ID,WVL_ID,WVL_ID,ID,GEOVISIA_I,WVL_ID,ID
2,3,BORNAME,-1,-1,BORNAME,-1,-1,BORNAME
3,3,ONDERDEELT,ONDERDEELT,ONDERDEELT,ONDERDEELT,-1,Onderdeelt,ONDERDEELT
4,3,WEGVAKNUMM,WEGVAK,WEGVAK,WEGVAKNUMM,-1,Wegvak,WEGVAKNUMM
5,3,ONDERDEELN,ONDERDEEL,ONDERDEEL,ONDERDEELN,-1,Onderdeel,ONDERDEELN
6,3,CLUSTERNUM,-1,-1,CLUSTERNUM,-1,Clusternum,CLUSTERNUM
7,1,GEBR_FUNTIE,GEBRUIKSFU,GEBRUIKSFU,GEBRUIKSFU,WEGFUNCTIE,Gebruiksfu,GEBRUIKSFU
8,1,WEGTYPE,WEGTYPE,WEGTYPE,WEGTYPE,WEGTYPEBES,Wegtype,WEGTYPE
9,1,BOVENSTE_V,VERHARDING,VERHARDING,BOVENSTE_V,TYPE_DETAI,Verharding,BOVENSTE_V


In [10]:
df_datavelden_units

,Prio,Dataveld,SI_unit,Borsele,Goes,Hulst,Kapelle,Middelburg,Noord-Beveland,Reimerswaal,Sluis,Terneuzen,Tholen,Veere,Vlissingen
0,1,Datum,yyyy-mm-dd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyy
1,1,Lengte,m,m,m,m,m,m,m,m,m,m,m,m,m
2,1,DiaBrHgt,m,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm
3,1,BOB,m,m,m,m,m,m,m,m,m,m,m,m,m
4,1,NAP,m,m,m,m,m,m,m,m,m,m,m,m,m
5,1,DatInsp,yyyy-mm-dd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyymmdd,yyyy


### Functie - Shape Files Leidingen Inladen
De .shp bestanden bevatten informatie over de leidingen zoals: lengte, breedte, straat etc. Een alternatieve databron zijn geopackages bestanden. Dit zou in de toekomst geimplementeerd kunnen worden.

Datetimes omzetten was erg problematisch. Veel try & excepts nodig om de data goed te kunnen parsen. Probleem was alleen bij begindatum 'BegDat' en niet bij de andere datums.

De eenheden worden waar nodig gestandaardiseerd naar SI. De variabelen diameter, breedte en hoogte blijven in mm.

In [11]:
def func_shapefile_lei_inladen(gemeente_naam,gemeente_lei_sf):
    # shapefile data van een gemeente inladen
    
    print('Shape Files Leidingen Inladen...',gemeente_naam)
    
    # Alle kolommen inladen
    #------------------------------------------------------------------------
    dict_gemeente_lei = {}

    for i in range(1,len(gemeente_lei_sf.fields)):

        col_name = gemeente_lei_sf.fields[i][0]
        col_fill = []

        for record_ in gemeente_lei_sf.iterRecords():

            col_fill.append(record_[i-1])

        dict_gemeente_lei[col_name] = col_fill

    df_gemeente_lei = pd.DataFrame(dict_gemeente_lei)
    
    col_coordinates_type = []
    col_coordinates = []
    for shape_i in gemeente_lei_sf.iterShapes():

        dict_i = shape_i.__geo_interface__
        col_coordinates_type.append(dict_i['type'])
        col_coordinates.append(dict_i['coordinates'])

    df_gemeente_lei['coordinates_type'] = col_coordinates_type
    df_gemeente_lei['coordinates_rd'] = col_coordinates
    print(df_gemeente_lei['coordinates_type'].value_counts())
    #------------------------------------------------------------------------
    
    
    if gemeente_naam == 'Vlissingen':
        del dict_rename_kolommen_lei['TYPE']
    else:
        pass
    
    # print(df_gemeente_lei.columns[0:50])
    df_gemeente_lei.rename(dict_rename_kolommen_lei,axis=1,inplace=True)
    
    # Duplicate kolommen verwijderen (Vlissingen)
    df_gemeente_lei = df_gemeente_lei.loc[:,~df_gemeente_lei.columns.duplicated()].copy()
#     print(df_gemeente_lei.columns[0:50])
    df_gemeente_lei_null = pd.DataFrame(columns=list_kolommen_lei)                 # Leeg DataFrame met alle kolommen
    df_gemeente_lei_compleet = pd.concat([df_gemeente_lei_null,df_gemeente_lei])
    df_gemeente_lei_compleet['Gemeente'] = gemeente_naam
    print(df_gemeente_lei_compleet['TypLei'][0:10])
    
    # Indien nodig BHOBJDC000 aanmaken
    if df_gemeente_lei_compleet['BHOBJDC000'].value_counts().empty:
        print('---BHOBJDC000 Ontbreekt---')
        
        try:
#             df_gemeente_lei_compleet['BHOBJDC000'] = ('LEI_' + df_gemeente_lei_compleet['GROEP'].astype(str) + '_' + 
#                                                       df_gemeente_lei_compleet['LeiNam'])
            df_gemeente_lei_compleet['BHOBJDC000'] = ('LEI_' + df_gemeente_lei_compleet['KnpNamB'].astype(str) + '_' +
                                                     df_gemeente_lei_compleet['KnpNamE'].astype(str))
            # Controle nodig om .0 in nieuwe BHOBJDC000 te verwijderen
            list_BHOBJDC000_schoon = []
            for index, row in df_gemeente_lei_compleet.iterrows():
                if row['BHOBJDC000'][-2:] == '.0':
                    list_BHOBJDC000_schoon.append(row['BHOBJDC000'][:-2])
                else:
                    list_BHOBJDC000_schoon.append(row['BHOBJDC000'])
            df_gemeente_lei_compleet['BHOBJDC000'] = list_BHOBJDC000_schoon
            print('---BHOBJDC000 Aangemaakt--- met GROEP')
        except(KeyError): 
#             df_gemeente_lei_compleet['BHOBJDC000'] = ('LEI_' + df_gemeente_lei_compleet['STELSTEL'].astype(str) + '_' +
#                                                      df_gemeente_lei_compleet['LeiNam'].astype(str))
            df_gemeente_lei_compleet['BHOBJDC000'] = ('LEI_' + df_gemeente_lei_compleet['KnpNamB'].astype(str) + '_' +
                                                     df_gemeente_lei_compleet['KnpNamE'].astype(str))
            
            
            # Controle nodig om .0 in nieuwe BHOBJDC000 te verwijderen
            list_BHOBJDC000_schoon = []
            for index, row in df_gemeente_lei_compleet.iterrows():
                if row['BHOBJDC000'][-2:] == '.0':
                    list_BHOBJDC000_schoon.append(row['BHOBJDC000'][:-2])
                else:
                    list_BHOBJDC000_schoon.append(row['BHOBJDC000'])
            df_gemeente_lei_compleet['BHOBJDC000'] = list_BHOBJDC000_schoon
            print('---BHOBJDC000 Aangemaakt--- met STELSTEL')
    else:
        pass
    
    df_gemeente_lei_compleet = df_gemeente_lei_compleet[list_kolommen_lei].copy()  # Alleen relevante kolommen
    list_kolommen_lei_gemeente = list(df_gemeente_lei_compleet.columns)            # Lijst van kolommen
    
    
    print('Oorspronkelijke datum:',df_gemeente_lei_compleet['BegDat'].iloc[0])
    
    # Datetimes omzetten
    #---------------------------------------------------------------------------------------------------------
    # Probleem met bytes: b'********', gebruik coerce om inlezen te forceren
    
    # BegDat
    try:
        df_gemeente_lei_compleet['BegDat_datum'] = pd.to_datetime(
            df_gemeente_lei_compleet['BegDat'],format='%Y%m%d')
    except(TypeError):
        df_gemeente_lei_compleet['BegDat_datum'] = pd.to_datetime(
            df_gemeente_lei_compleet['BegDat'],errors='coerce',format='%Y%m%d')
    except(ValueError):
        try:
            df_gemeente_lei_compleet['BegDat_datum'] = pd.to_datetime(
                df_gemeente_lei_compleet['BegDat'],format='%Y-%m-%d')
        except(TypeError):
            df_gemeente_lei_compleet['BegDat_datum'] = pd.to_datetime(
                df_gemeente_lei_compleet['BegDat'],errors='coerce',format='%Y-%m-%d')
        except(ValueError):
            try:
                df_gemeente_lei_compleet['BegDat_datum'] = pd.to_datetime(
                    df_gemeente_lei_compleet['BegDat'],format='%d%m%Y')
            except(TypeError):
                df_gemeente_lei_compleet['BegDat_datum'] = pd.to_datetime(
                    df_gemeente_lei_compleet['BegDat'],errors='coerce',format='%d%m%Y')
            
                
    # DatAanpGep
    try:
        df_gemeente_lei_compleet['DatAanpGep_datum'] = pd.to_datetime(df_gemeente_lei_compleet['DatAanpGep'],
                                                                      errors='coerce',format='%Y%m%d')
    except(ValueError,TypeError):
        df_gemeente_lei_compleet['DatAanpGep_datum'] = pd.to_datetime(df_gemeente_lei_compleet['DatAanpGep'],
                                                                      errors='coerce',format='%Y-%m-%d') 
    # DatInsp
    try:
        df_gemeente_lei_compleet['DatInsp_datum'] = pd.to_datetime(df_gemeente_lei_compleet['DatInsp'],
                                                                   errors='coerce',format='%Y%m%d')
    except(ValueError,TypeError):
        df_gemeente_lei_compleet['DatInsp_datum'] = pd.to_datetime(df_gemeente_lei_compleet['DatInsp'],
                                                                   errors='coerce',format='%Y-%m-%d')
    #-----------------------------------------------------------------------------------------------------------
    
    print('Nieuwe datum:',df_gemeente_lei_compleet['BegDat_datum'].iloc[0])
    print('Lengtes voor conversie',df_gemeente_lei_compleet['LenLei'][0:10])
    
    # Eenheden standaardiseren naar SI
    # In ieder geval bij Borsele een inconsistentie tussen de eenheden van .shp en .csv wanneer je data van
    # Data portaal Zeeland download
    int_lenlei = len(str(math.ceil(df_gemeente_lei_compleet['LenLei'].median())))         # Naar boven afgeronde mediaan
    int_br = len(str(math.ceil(df_gemeente_lei_compleet['Br'].median())))                 # Naar boven afgeronde mediaan
    int_hgt = len(str(math.ceil(df_gemeente_lei_compleet['Hgt'].median())))               # Naar boven afgeronde mediaan
    int_bobb = len(str(math.ceil(df_gemeente_lei_compleet['BobB'].median())))
    int_bobe = len(str(math.ceil(df_gemeente_lei_compleet['BobE'].median())))
    
    print('Aantal cijfers voor komma LenLei:',int_lenlei)
    print('Aantal cijfers voor komma Br:    ',int_br)
    print('Aantal cijfers voor komma Hgt:   ',int_hgt)
    print('Aantal cijfers voor komma BobB:  ',int_bobb)
    print('Aantal cijfers voor komma BobE:  ',int_bobe)
    
    dict_eenheden_gemeente = dict(zip(df_datavelden_units['Dataveld'],df_datavelden_units[gemeente_naam]))
    dict_conversiefactor = {'m': 1,
                            'dm': 10,
                            'cm': 100,
                            'mm': 1000}
    
    # Conversies toepassen
    # Diameter wordt opzettelijk niet geconverteerd, omdat BAJA classificatie in mm gebeurt
    df_gemeente_lei_compleet['LenLei'] = (df_gemeente_lei_compleet['LenLei']/
                                          dict_conversiefactor[dict_eenheden_gemeente['Lengte']])
    df_gemeente_lei_compleet['Br'] = (df_gemeente_lei_compleet['Br']/
                                      dict_conversiefactor[dict_eenheden_gemeente['DiaBrHgt']])
    df_gemeente_lei_compleet['Hgt'] = (df_gemeente_lei_compleet['Hgt']/
                                       dict_conversiefactor[dict_eenheden_gemeente['DiaBrHgt']])
    df_gemeente_lei_compleet['BobB'] = (df_gemeente_lei_compleet['BobB']/
                                        dict_conversiefactor[dict_eenheden_gemeente['BOB']])
    df_gemeente_lei_compleet['BobE'] = (df_gemeente_lei_compleet['BobE']/
                                        dict_conversiefactor[dict_eenheden_gemeente['BOB']])
    
    print('Shape Files Leidingen Ingeladen...')
    print(df_gemeente_lei_compleet.shape[0])
    print(df_gemeente_lei_compleet.columns)
    return df_gemeente_lei_compleet
    

### Functie - .ribx Inladen
.ribx bestanden zijn in principe XML bestanden. Element Tree wordt gebruikt om de data in te lezen.

Bestanden van gemeente Vlissingen hebben soms de tags 'AAC', 'AAE' en 'AAG' niet. Soms zijn de tags wel aanwezig, maar zit er geen informatie in.

Per inspectie binnen een streng (ZC) zijn de letters (bijv. A, B etc.) niet allemaal aanwezig. In deze functie wordt een soort alfabet iteratie gedaan om alles te pakken.

In [12]:
def func_ribx_inladen(filename,root):
    print('---Start .ribx bestanden inladen---')
    
    list_dfs = []
    i_count = 0
    list_letters = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P']
    
    for element_strengdata in root.findall('ZB_A'):
        
        dict_strengdata = {}
#         print(element_strengdata)
        for sub_element in element_strengdata:
            
            list_tags = ['AAC','AAE','AAG']
            
            if sub_element.tag in list_tags:

                for Point in sub_element.findall('{http://www.opengis.net/gml}Point'):
                    str_point = Point.findtext('{http://www.opengis.net/gml}pos')      # Punt inlezen
                    str_point_comma = str_point.replace(' ',',')                       # Spatie met comma vervangen
                    list_point = str_point_comma.split(',')                            # List maken
                    list_point = list(filter(lambda x: x!= '', list_point))            # Verwijder leeg element (Vlissingen)
                    
                    if not list_point:                                                 # Controle lege list
                        list_point_float = [-999.999,-999.999]                         # In geval van geen data
                    elif len(list_point) == 1:
                        list_point_float = [-999.999,-999.999]                         # Maar een element niet twee
                    else:
#                         print(list_point)
                        list_point_float = [float(list_point[0]),float(list_point[1])] # Omzetten naar getal

                    dict_strengdata[sub_element.tag] = str(list_point_float)           # 1 dimensie van dictionary behouden
                    
                    
            elif sub_element.tag == 'AXY':

                for LineString in sub_element.findall('{http://www.opengis.net/gml}LineString'):

                    list_coordinates = []

                    for pos in LineString.findall('{http://www.opengis.net/gml}pos'):

                        str_point = pos.text                                                   # Punt inlezen
                        str_point_comma = str_point.replace(' ',',')                           # Spatie met comma vervangen
                        list_point = str_point_comma.split(',')                                # List maken
                        list_coordinates.append((float(list_point[0]),float(list_point[1])))   # Omzetten naar getal

                    dict_strengdata[sub_element.tag] = str(list_coordinates)                   # 1 dimensie dictionary

            else:
                dict_strengdata[sub_element.tag] = sub_element.text


            if sub_element.tag == 'ZC':
                i_count += 1
                
                # Dictionary leegmaken, niet alle ZCs hebben dezelfde letters
                #------------------------------------
                for letter in list_letters:
                    str_letter = 'ZC-' + letter
                    dict_strengdata[str_letter] = ''
                #------------------------------------

                # Vul subelementen van ZC
                #--------------------------------------------------------------
                for sub_sub_element in sub_element:

                    for letter in list_letters:
                        if sub_sub_element.tag == letter:
                            str_letter = 'ZC-' + letter
                            dict_strengdata[str_letter] = sub_sub_element.text
                        else:
                            pass
                #---------------------------------------------------------------

                df_strengdata = pd.DataFrame(dict_strengdata,index=[i_count])
                list_dfs.append(df_strengdata)
#                 print('df_strengdata.empty:',df_strengdata.empty)
#                 print('')
            else:
                pass
    
    df_strengdata_concat = pd.concat(list_dfs)
#     print('Size strengdata_concat',df_strengdata_concat.shape)
    df_ribx = df_strengdata_concat.copy()
    df_ribx = df_ribx.loc[df_ribx['ZC-A'] != 'BXA'].reset_index(drop=True)
    df_ribx['NamRibx'] = filename[:-5]                                        # Naam bestand
    
    return df_ribx

### Functie - .ribx verwerken

.ribx bestanden worden met de bovenstaande functie ingeladen. Vervolgens worden de kolomnamen vervangen. Er wordt nog een unieke strengnaam geproduceerd. Sommige gemeenten gebruiken steeds dezelfde strengnamen per inspectiesessie/dag. Bestandnaam toevoegen om te garanderen dat het uniek is.

In [13]:
def func_ribx_processing(directory_gemeente_ribx):
    # ParseError bij Borsele, 'latin1', 'utf-8' en 'windows-1252' werken allemaal niet (helemaal)
    # & karakter is handmatig verwijderd uit bestand
    # Probleem bij Borsele
    # <AAA>051205-051206-1</AAA>
    # <AAA>121039-121039X-1</AAA>
    
    # ribx data per gemeente ophalen en inlezen
    os.chdir(directory_gemeente_ribx)
    
    # Error handling
    list_dfs_ribx_totaal = []
    for filename in os.listdir(directory_gemeente_ribx):
        parser = ET.XMLParser(encoding = 'utf-8')    # Standaard parser met utf-8
        
        filetype_ribx = filename[-5:].lower()
        filetype_txt = filename[-4:].lower()
        if filetype_ribx != '.ribx':
            if filetype_txt == '.txt':               # Werd voor een aangepaste .ribx gebruikt
                continue
            else:
                continue
        else:
            pass
        
        # Parsing met utf-8 of latin1
        try:
            tree = ET.parse(filename,parser=parser)
            root = tree.getroot()
        except:
            print('--- PARSE ERROR ELEMENT TREE ---')
            print('Bestand met parse error:',filename)
            try:
                parser = ET.XMLParser(encoding = 'utf-8')
                tree = ET.parse(filename,parser=parser)
                root = tree.getroot()
                print('Parse met utf-8')
            except:
                try:
                    parser = ET.XMLParser(encoding = 'latin1')
                    tree = ET.parse(filename,parser=parser)
                    root = tree.getroot()
                    print('Parse met latin1')
                except:
                    print('Bestand met parse error overslaan:',filename)
                    continue
            
        # Het kan zijn dat een .ribx bestand geen metingen heeft
        if not root.findall('ZB_A'):
            continue
        else:
            pass
        
        try:
            df_ribx_bestand = func_ribx_inladen(filename,root)
        except(ValueError):
            print('Leeg ribx DF:',filename)
            continue
        
        list_dfs_ribx_totaal.append(df_ribx_bestand)
        
    df_ribx_totaal = pd.concat(list_dfs_ribx_totaal)
    df_ribx_totaal = df_ribx_totaal.reset_index(drop=True)
    print('kolommen df_ribx_totaal')
    print(df_ribx_totaal.columns[0:50])
    dict_rename_cols = {'AAA': 'streng',
                    'AAD': 'knooppunt_meting_begin',
                    'AAE': 'knooppunt_rd_begin',
                    'AAF': 'knooppunt_meting_eind',
                    'AAG': 'knooppunt_rd_eind',
                    'AAJ': 'straat',
                    'AAK': 'inspectierichting',
                    'AAL': 'soort_locatie',
                    'AAM': 'opdrachtgever',
                    'AAN': 'woonplaats',
                    'AAO': 'wijk',
                    'AAP': 'naam_rioolstelsel',
                    'AAQ': 'grondeigendom',
                    'ABA': 'norm',
                    'ABC': 'referentiepunt_langsrichting',
                    'ABE': 'inspectiemethode',
                    'ABF': 'inspectiedatum',
                    'ABG': 'inspectietijd',
                    'ABS': 'videobestandsnaam',
                    'ACA': 'vorm',
                    'ACB': 'pijp_hoogte',
                    'ACC': 'pijp_breedte',
                    'ACD': 'materiaal',
                    'ACE': 'lining_soort',
                    'ACF': 'lining_materiaal',
                    'ADE': 'algemene_opmerking',
                    'AXA': 'tekeningnummer',
                    'AXB': 'verharding_wegdek',
                    'AXC': 'leiding_ontbreekt_in_opdracht',
                    'AXD': 'leiding_niet_behandeld',
                    'AXE': 'wanddikte',
                    'AXF': 'verbinding_type',
                    'AXG': 'nap_bob_beginpunt',
                    'AXH': 'nap_bob_eindpunt',
                    'AXY': 'leidingcurve',
                    'ZC-A': 'classificatie_hoofdcode',                    # Hoofdcode
                    'ZC-B': 'classificatie_kar1',                         # Karakterisering 1
                    'ZC-C': 'classificatie_kar2',                         # Karakterisering 2
                    'ZC-D': 'classificatie_kwantificering1',              # Kwantificering 1
                    'ZC-E': 'classificatie_kwantificering2',              # Kwantificering 2
                    'ZC-F': 'classificatie_text',                         # Opmerkingen
                    'ZC-G': 'classificiate_plaats_omtrek1',               # Plaats op de omtrek 1
                    'ZC-H': 'classificatie_plaats_omtrek2',               # Plaats op de omtrek 2
                    'ZC-I': 'classificatie_afstand',                      # Axiale of verticale plaats (afstand)
                    'ZC-J': 'classificatie_meelopende_schade',            # Code voor een doorlopende hoofdcode
                    'ZC-M': 'classificatie_foto',                         # Referentiefoto
                    'ZC-N': 'classificatie_tijd',                         # Referentievideo
                    'ZC-O': 'classificatie_kar3'}                         # Karakterisering 3
    
    df_inspectiedata = df_ribx_totaal.rename(columns=dict_rename_cols)
    df_inspectiedata = df_inspectiedata.loc[df_inspectiedata['classificatie_hoofdcode'] != 'BXA'].reset_index(drop=True)
    df_inspectiedata['streng_uniek'] = (df_inspectiedata['NamRibx'] + '_' + df_inspectiedata['streng'])
    print(df_inspectiedata.columns[0:50])
    return df_inspectiedata

### Functie - Coordinaten Opvissen

Begin- en eindcoordinaten uit lijst halen om .shp met .ribx te combineren.

In [14]:
def func_coordinates_rd_split(list_coordinates_rd):
    
    coordinate_knpb = list_coordinates_rd[0]
    coordinate_knpe = list_coordinates_rd[-1]
    
    list_coordinate_knpb = [coordinate_knpb[0],coordinate_knpb[-1]]
    list_coordinate_knpe = [coordinate_knpe[0],coordinate_knpe[-1]]
    
    list_coordinate_knpb_str = str(list_coordinate_knpb)
    list_coordinate_knpe_str = str(list_coordinate_knpe)
    
    return list_coordinate_knpb_str, list_coordinate_knpe_str

### Functie - .ribx & .shp Combineren

Beheerobject en leidingnaam toevoegen aan de inspectiedata (.ribx). Beide dfs zijn nodig door randgevallen. De meeste kunnen op basis van knooppuntnaam.


In [15]:
def func_koppeling_ribx_shp(df_gemeente_leidingen,df_gemeente_inspectiedata):
    # Een deel van .ribx Reimerswaal heeft alleen beginknooppunt, maar GEEN eindknooppunt
    # Data wordt van de .shp bestanden gepakt en gekoppeld met de data van .ribx
    # Dictionary iteration in plaats van .iterrows()
    
    list_BHOBJDC000 = []
    list_LeiNam = []
    list_controle_BHOBJDC000 = []
    list_controle_LeiNam = []
    list_controle_inspectiedata_meting = []
    list_controle_inspectiedata_rd = []
    list_controle_leidingen_meting = []
    list_controle_leidingen_rd = []
    
    dict_gemeente_inspectiedata = df_gemeente_inspectiedata.to_dict('records')
    
    for row in dict_gemeente_inspectiedata:
        
        # Middelburg en Hulst hebben geen RD coordinaten bij de inspecties
        try:
            knooppunt_naam_begin = row['knooppunt_meting_begin']
            knooppunt_naam_eind = row['knooppunt_meting_eind']
            knooppunt_rd_begin = row['knooppunt_rd_begin']
            knooppunt_rd_eind = row['knooppunt_rd_eind']
        except(KeyError):
            knooppunt_naam_begin = row['knooppunt_meting_begin']
            knooppunt_naam_eind = row['knooppunt_meting_eind']
            knooppunt_rd_begin = -9999
            knooppunt_rd_eind = -9999
        
        # Filter op basis van knooppuntnaam
        df_leiding_filter_knpnam = df_gemeente_leidingen.loc[
            ((df_gemeente_leidingen['KnpNamB'] == knooppunt_naam_begin) |
             (df_gemeente_leidingen['KnpNamB'] == knooppunt_naam_eind)) &
            ((df_gemeente_leidingen['KnpNamE'] == knooppunt_naam_begin) |
             (df_gemeente_leidingen['KnpNamE'] == knooppunt_naam_eind))].reset_index(drop=True)

        # Filter op basis van Rijksdriehoekcoördinaten
        df_leiding_filter_rd = df_gemeente_leidingen.loc[
            ((df_gemeente_leidingen['knooppunt_rd_begin'] == knooppunt_rd_begin) |
             (df_gemeente_leidingen['knooppunt_rd_begin'] == knooppunt_rd_eind)) &
            ((df_gemeente_leidingen['knooppunt_rd_eind'] == knooppunt_rd_begin) |
             (df_gemeente_leidingen['knooppunt_rd_eind'] == knooppunt_rd_eind))].reset_index(drop=True)

        # Beide DataFrames zijn leeg
        if df_leiding_filter_knpnam.empty & df_leiding_filter_rd.empty:
            list_BHOBJDC000.append('GEEN MATCH')
            list_LeiNam.append('GEEN MATCH')
            
        # Geen link tussen knooppuntnaam in .shp en .ribx
        elif df_leiding_filter_knpnam.empty:
            if df_leiding_filter_rd.empty == False:
                list_BHOBJDC000.append(df_leiding_filter_rd['BHOBJDC000'].iloc[0])
                list_LeiNam.append(df_leiding_filter_rd['LeiNam'].iloc[0])
                list_controle_BHOBJDC000.append(df_leiding_filter_rd['BHOBJDC000'].iloc[0])
                list_controle_LeiNam.append(df_leiding_filter_rd['LeiNam'].iloc[0])
                list_controle_inspectiedata_meting.append(str([row['knooppunt_meting_begin'],row['knooppunt_meting_eind']]))
                list_controle_inspectiedata_rd.append(str([knooppunt_rd_begin,knooppunt_rd_eind]))
                list_controle_leidingen_meting.append('GEEN MATCH KNOOPPUNTEN')
                list_controle_leidingen_rd.append(str([df_leiding_filter_rd['knooppunt_rd_begin'].iloc[0],
                                                      df_leiding_filter_rd['knooppunt_rd_eind'].iloc[0]]))
            else:
                list_BHOBJDC000.append('GEEN MATCH')
                list_LeiNam.append('GEEN MATCH')
        else:
            if df_leiding_filter_knpnam.empty == False:
                list_BHOBJDC000.append(df_leiding_filter_knpnam['BHOBJDC000'].iloc[0])
                list_LeiNam.append(df_leiding_filter_knpnam['LeiNam'].iloc[0])
                list_controle_BHOBJDC000.append(df_leiding_filter_knpnam['BHOBJDC000'].iloc[0])
                list_controle_LeiNam.append(df_leiding_filter_knpnam['LeiNam'].iloc[0])
                list_controle_inspectiedata_meting.append(str([row['knooppunt_meting_begin'],row['knooppunt_meting_eind']]))
                list_controle_inspectiedata_rd.append(str([knooppunt_rd_begin,knooppunt_rd_eind]))
                list_controle_leidingen_meting.append(str([df_leiding_filter_knpnam['KnpNamB'].iloc[0],
                                                          df_leiding_filter_knpnam['KnpNamE'].iloc[0]]))
                list_controle_leidingen_rd.append('MATCH LUKTE MET KNOOPPUNTEN')
            else:
                list_BHOBJDC000.append('GEEN MATCH')
                list_LeiNam.append('GEEN MATCH')
    
    df_gemeente_inspectiedata['BHOBJDC000'] = list_BHOBJDC000
    df_gemeente_inspectiedata['LeiNam'] = list_LeiNam
    
    # Aantal .ribx strengen zonder een match
    num_geenmatch = len(df_gemeente_inspectiedata.loc[
        df_gemeente_inspectiedata['LeiNam'] == 'GEEN MATCH'].groupby(by=['streng_uniek']).size())
    num_match = len(df_gemeente_inspectiedata.loc[
        df_gemeente_inspectiedata['LeiNam'] != 'GEEN MATCH'].groupby(by=['streng_uniek']).size())
    num_leidingen = len(df_gemeente_leidingen['LeiNam'].unique())
    print('Aantal .ribx strengen zonder een match:',num_geenmatch)
    print('Aantal .ribx strengen met een match:   ',num_match)
    print('Aantal leidingen:                      ',num_leidingen)
    
    # Een streng per leiding kiezen
    df_gb = pd.DataFrame(df_gemeente_inspectiedata.groupby(
        by=['BHOBJDC000','streng_uniek','inspectiedatum']).size(),columns=['aantal']).reset_index()
    df_gb = df_gb.loc[df_gb['BHOBJDC000'] != 'GEEN MATCH'].reset_index(drop=True)       # Verwijder leidingen zonder match
    
    df_gemeente_inspectiedata_mm = df_gemeente_inspectiedata.copy()   # DF met strengen met meerder inspecties
    list_streng_uniek_recent = []                                     # Unieke strengen met een match
    
    # Tijdelijke oplossing met for-loop
    array_BHOBJDC000 = df_gb['BHOBJDC000'].unique()
    for BHOBJDC000 in array_BHOBJDC000:
        
        df_BHOBJDC000 = df_gb.loc[df_gb['BHOBJDC000'] == BHOBJDC000].reset_index(drop=True)
        df_BHOBJDC000.sort_values(by=['inspectiedatum'],ascending=False,inplace=True)
        list_streng_uniek_recent.append(df_BHOBJDC000['streng_uniek'].iloc[0])
        
    df_gemeente_inspectiedata = df_gemeente_inspectiedata.loc[
        df_gemeente_inspectiedata['streng_uniek'].isin(list_streng_uniek_recent)].reset_index(drop=True)
    
    print('Aantal streng_uniek na selectie meest recent:',len(list_streng_uniek_recent))
    
    return df_gemeente_inspectiedata, df_gemeente_inspectiedata_mm

### Functie - Materiaal Vervangen
In de .ribx bestanden zitten codes voor het materiaaltype. Deze moeten vervangen worden. De meeste leidingen bestaan uit beton of polyvinylchloride. Andere typen komen in mindere mate voor, maar dit verschilt per gemeente.

Gemeente Vlissingen heeft geen kolom met verbinding.

In [16]:
def func_materiaal_vervangen(df_gemeente_leidingen,df_gemeente_inspectiedata):
    
    # Controle ontbrekende data
    df_gemeente_inspectiedata['duplicate'] = df_gemeente_inspectiedata.duplicated(subset=['streng_uniek'],keep='first')
    df_duplicated = df_gemeente_inspectiedata.loc[df_gemeente_inspectiedata['duplicate'] == False].reset_index(drop=True)
    
    print('len inspectiedata',df_gemeente_inspectiedata.shape[0])
    print('len duplicated',df_duplicated.shape[0])
    print('Materiaal df_duplicated:        ',df_duplicated['materiaal'].value_counts())
    print('Pijp breedtes df_duplicated:    ',df_duplicated['pijp_breedte'].value_counts(dropna=False))
    print('Diameters df_gemeente_leidingen:',df_gemeente_leidingen['Diam'].value_counts(dropna=False))
    
    # Materiaal
    df_gemeente_inspectiedata['Mat'] = df_gemeente_inspectiedata['materiaal'].copy()       # Nieuwe kolom met andere naam
    
    dict_mat = dict(zip(df_gemeente_leidingen['BHOBJDC000'],df_gemeente_leidingen['Mat'])) # Wordt niet gebruikt
    
    dict_materiaal_vervangen = {'AA': 'Asbestcement',
                                'AB': 'Bitumen',
                                'AC': 'Bitumen-houtvezel composiet',
                                'AD': 'Metselwerk (baksteen)',
                                'AE': 'Klei',
                                'AF': 'Cementmortel',
                                'AG': 'Beton',
                                'AH': 'Gewapend beton',
                                'AI': 'Gespoten beton',
                                'AJ': 'Betonnen segmenten',
                                'AK': 'Vezelcement',
                                'AL': 'Glasvezel versterkte kunststof',
                                'AM': 'Gietijzer',
                                'AN': 'Grijs gietijzer',
                                'AO': 'Nodulair gietijzer',
                                'AP': 'Staal',
                                'AR': 'Metselwerk (bepleisterd)',
                                'AS': 'Metselwerk (onbepleisterd)',
                                'AT': 'Epoxy',
                                'AU': 'Polyester',
                                'AV': 'Polyetheen',
                                'AW': 'Polypropyleen',
                                'AX': 'Polyvinylchloride',
                                'AY': 'Unidentified type of plastics',
                                'AZ': 'Unidentified material',
                                'Z': 'Anders',
                                'ZZ': 'Anders'}            # Noord-Beveland had 8 leidingen met 'ZZ'
    
    df_gemeente_inspectiedata['Mat'].replace(dict_materiaal_vervangen,inplace=True)
    
    # Gemeente Vlissingen heeft geen verbinding
    try:
        dict_vrb = dict(zip(df_gemeente_leidingen['BHOBJDC000'],df_gemeente_leidingen['Vrb']))   # Verbinding
        df_gemeente_inspectiedata['Vrb'] = df_gemeente_inspectiedata['BHOBJDC000'].map(dict_vrb)
    except(KeyError):
        df_gemeente_inspectiedata['Vrb'] = np.nan
        df_gemeente_leidingen['Vrb'] = np.nan                           # Nodig voor ML, consistentie met andere gemeenten
        print('GEEN Vrb (verbinding) in inspectiedata, leeg Vrb kolom aangemaakt')
        
    print('Rijen df_gemeente_inspectiedata voor dropna:',df_gemeente_inspectiedata.shape[0])    
    df_gemeente_inspectiedata = df_gemeente_inspectiedata.dropna(subset=['classificatie_hoofdcode'])
    df_gemeente_inspectiedata = df_gemeente_inspectiedata.reset_index(drop=True)
    print('Rijen df_gemeente_inspectiedata na dropna:  ',df_gemeente_inspectiedata.shape[0])
    
    return df_gemeente_inspectiedata

### Functie - Classificatie Strengen

In deze functie worden de metingen in de .ribx strengen (inspectie per leiding) geclassificeerd.

Gemeente Vlissingen heeft 17009 BAJC incidenten. 14818 hebben een hoekverdraaiing van 45 graden. 1706 hebben een verdraaiing van 1 graad en 410 van 2. De rest zijn tussen de 3 en 44.

In [17]:
def func_classificatie_strengen(df_gemeente_inspectiedata,
                               dict_BAJA_mat, dict_BAJA_klasse, dict_BAJA_gd_koga,
                               list_baja_beton_mof, list_baja_beton_vaar, list_baja_kunstof, list_baja_overig,
                               list_pijp_breedtes):
    # Dictionary iteration in plaats van .iterrows()
    
    dict_iter = df_gemeente_inspectiedata.to_dict('index')    # Dictionary van de rijen
    list_keys = list(dict_iter.keys())
    
    # Tellingen voor fouten
    count_fout_hoofdcode = 0
    count_fout_codenietinbestand = 0
    count_fout_geenkwantificering1 = 0
    count_fout_geenkwantificering2 = 0
    count_fout_kwantificeringistekst = 0
    count_fout_dataframeklasseindelingleeg = 0
    count_fout_bajageenpijpbreedte = 0
    count_fout_bajageenmateriaal = 0
    
    # Lists voor nieuwe DataFrame
    list_klasse_indeling = []
    list_code = []
    list_hc = []
    list_kar1 = []
    list_kar2 = []
    list_kar3 = []
    list_classificatie_kwantificering1 = []
    list_classificatie_kwantificering2 = []
    
    for index in list_keys:
        
        row = dict_iter[index]

        # DataFrame van streng (gebruik 'streng_uniek')
        df_streng = df_gemeente_inspectiedata.loc[
            df_gemeente_inspectiedata['streng_uniek'] == 
            df_gemeente_inspectiedata['streng_uniek'].loc[index]].reset_index(drop=True)

        code_hckar1kar2 = (row['classificatie_hoofdcode'] + row['classificatie_kar1'] + row['classificatie_kar2'])
        code_hckar1 = (row['classificatie_hoofdcode'] + row['classificatie_kar1'])
        
        list_code.append(code_hckar1kar2)
        list_hc.append(row['classificatie_hoofdcode'])
        list_kar1.append(row['classificatie_kar1'])
        list_kar2.append(row['classificatie_kar2'])
        list_kar3.append(row['classificatie_kar3'])
        list_classificatie_kwantificering1.append(row['classificatie_kwantificering1'])
        list_classificatie_kwantificering2.append(row['classificatie_kwantificering2'])

        # Hoofdcode
        df_code_hc = df_classificatie_leiding.loc[
            (df_classificatie_leiding['Hoofdcode'] == row['classificatie_hoofdcode'])].reset_index(drop=True)

        df_code_hckar1 = df_classificatie_leiding.loc[
            (df_classificatie_leiding['Hoofdcode'] == row['classificatie_hoofdcode']) &
            (df_classificatie_leiding['Karakterisering 1 code'] == row['classificatie_kar1'])
        ].reset_index(drop=True)

        # DataFrame maken met totale (specifieke) code
        df_code_hckar1kar2 = df_code_hc.loc[(df_code_hc['code_hckar1kar2'] == code_hckar1kar2)].reset_index(drop=True)
        
        # Codes die beginnen met een A uitfilteren
        if code_hckar1kar2[0] == 'A':
            list_klasse_indeling.append('n.v.t.')
            continue
        else:
            pass
        
        
        
        # Eerste indeling (klasse niet van toepassing)
        #-----------------------------------------------------
        if df_code_hc.empty:
            count_fout_hoofdcode += 1
            list_klasse_indeling.append('n.v.t.')                  # Hoofdcode matcht niet met beschikbare opties
            print(code_hckar1kar2, 'n.v.t. - HOOFDCODE BESTAAT NIET')
            continue
        elif df_code_hc['Klasse-indeling'].iloc[0] == 'G':         # 'G' is leeg
            list_klasse_indeling.append('n.v.t.')
            continue
        else:
            pass
        #-----------------------------------------------------
        
        # Tweede indeling
        # Geen extra getal (classificatie_kwantificering1) nodig voor klasse
        #---------------------------------------------------------------------------------------------------------------
        if df_code_hc.shape[0] == 1:
            # Eén optie mogelijk met hc
            value_ = df_code_hc['Klasse-indeling'].iloc[0]
            list_klasse_indeling.append(value_)
            continue
        elif ((row['classificatie_kwantificering1'] == '') & 
              (row['classificatie_kwantificering2'] == '') &
              (df_code_hckar1kar2.shape[0] == 1)):
            # Eén optie mogelijk met kar12
            value_ = df_code_hckar1kar2['Klasse-indeling'].iloc[0]  # Geen kwantificering in data
            list_klasse_indeling.append(value_)
            continue
        else:
            pass
        
        # Derde Indeling
        #----------------------------------------------------------------------------------------------------------------
        if row['classificatie_kar1'] == '':                                           # Karakter vervangen
            str_new_kar1 = '#'
        else:
            str_new_kar1 = row['classificatie_kar1']                                  

        if row['classificatie_kar2'] == '':                                           # Karakter vervangen
            str_new_kar2 = '#'
        else:
            str_new_kar2 = row['classificatie_kar2']                                  

        if row['classificatie_kar3'] == '':                                           # Karakter vervangen
            str_new_kar3 = '#'
        else:
            str_new_kar3 = row['classificatie_kar3']
        
        
        list_baj = ['BAJA','BAJC']                                                    # BAJB kan wel geclassificeerd worden
        if code_hckar1kar2 in list_baj:                                               # BAJA wordt later verwerkt
            pass
        elif df_code_hc.shape[0] > 1:                                                 # Meerdere hoofdcode opties
            
            df_klasse_selectie = df_code_hc.loc[
                (df_code_hc['Karakterisering 1 code'] == str_new_kar1) &
                (df_code_hc['Karakterisering 2 code'] == str_new_kar2) &
                (df_code_hc['Karakterisering 3 (RibX) code'] == str_new_kar3)].reset_index(drop=True)
            
            if df_klasse_selectie.empty:
                count_fout_codenietinbestand += 1
                list_klasse_indeling.append('n.v.t. - CODE NIET IN BESTAND')
                continue
            elif df_klasse_selectie.shape[0] == 1:       # Maar één optie
                list_klasse_indeling.append(df_klasse_selectie['Klasse-indeling'].iloc[0])
                continue
            else:
                pass

            kwantificering_waarde = -9999
            
            # Controle of kwantificering 1 en/of 2 leeg is
            if (row['classificatie_kwantificering1'] == '') & (row['classificatie_kwantificering2'] == ''):
                count_fout_geenkwantificering1 += 1
                count_fout_geenkwantificering2 += 1
                list_klasse_indeling.append('n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING')
                print(code_hckar1kar2,row['classificatie_kwantificering1'],row['classificatie_kwantificering2'],
                     'n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING')
                continue
            # kwantificering 1 is niet leeg, kwantificering 2 is wel leeg
            elif (row['classificatie_kwantificering1'] != '') & (row['classificatie_kwantificering2'] == ''):
                count_fout_geenkwantificering2 += 1
                kwantificering_waarde = row['classificatie_kwantificering1']
            # kwantificering 1 is wel leeg, kwantificering 2 is niet leeg
            elif (row['classificatie_kwantificering1'] == '') & (row['classificatie_kwantificering2'] != ''):
                count_fout_geenkwantificering1 += 1
                kwantificering_waarde = row['classificatie_kwantificering2']
            elif (row['classificatie_kwantificering1'] != '') & (row['classificatie_kwantificering2'] != ''):
                kwantificering_waarde = row['classificatie_kwantificering1']
            else:
                pass
            
            
            # In sommige rijen is de kwantificering niet een getal maar tekst. Meestal een getal met een punt erachter '.'
            # In gemeente Reimerswaal zijn er ook leidingen met een decimaal.
            try:
                int(kwantificering_waarde)
            except(ValueError):
                try:
                    float(kwantificering_waarde)
                except(ValueError):
                    count_fout_kwantificeringistekst += 1
                    print(code_hckar1kar2,row['classificatie_kwantificering1'],'n.v.t. - KWANTIFICERING IS TEKST')
                    list_klasse_indeling.append('n.v.t. - KWANTIFICERING IS TEKST')
                    continue
            
            df_klasse_selectie['Klassegrens: \ngroter dan >'] = pd.to_numeric(
                df_klasse_selectie['Klassegrens: \ngroter dan >'],errors='coerce')
            df_klasse_selectie['Klassegrens: \nkleiner of gelijk aan ≤'] = pd.to_numeric(
                df_klasse_selectie['Klassegrens: \nkleiner of gelijk aan ≤'],errors='coerce')
            
            # Sommige kwantificering waarden zijn 0. 0 kan niet groter dan 0
            if float(kwantificering_waarde) == 0:                     # Soms is een waarde 0
                value_klasse = 1
                list_klasse_indeling.append(value_klasse)
                continue
            elif float(kwantificering_waarde) > 100:                  # Soms is een waarde > 100
                value_klasse = 5
                list_klasse_indeling.append(value_klasse)
                continue
            else:
                pass
            
            df_klasse_indeling = df_klasse_selectie.loc[
                (df_klasse_selectie['Klassegrens: \ngroter dan >'] <= float(kwantificering_waarde)) &
                (df_klasse_selectie['Klassegrens: \nkleiner of gelijk aan ≤'] >= float(kwantificering_waarde))
                ].reset_index(drop=True)
                
            try:
                list_klasse_indeling.append(df_klasse_indeling['Klasse-indeling'].iloc[0])
            except(IndexError):
                print(df_klasse_selectie.shape)
                print(df_klasse_indeling.shape)
                print(kwantificering_waarde)
                print(float(kwantificering_waarde))
                count_fout_dataframeklasseindelingleeg += 1
                # row classificatie kwantificering1 hier behouden
                print(code_hckar1kar2,row['classificatie_kwantificering1'],'n.v.t. - DATAFRAME KLASSE INDELING LEEG')
                list_klasse_indeling.append('n.v.t. - DATAFRAME KLASSE INDELING LEEG')
            continue
            
        else:
            pass
        #----------------------------------------------------------------------------------------------------------------    
        
        # BAJA
        if code_hckar1kar2 == 'BAJA':                                           # Code van waarneming in .ribx 
            
            kwantificering_waarde = -9999
            
            # Controle of kwantificering 1 en/of 2 leeg is
            # kwantificering 1 en kwantificering 2 zijn leeg
            if (row['classificatie_kwantificering1'] == '') & (row['classificatie_kwantificering2'] == ''):
                count_fout_geenkwantificering1 += 1
                count_fout_geenkwantificering2 += 1
                list_klasse_indeling.append('n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING')
                print(code_hckar1kar2,row['classificatie_kwantificering1'],row['classificatie_kwantificering2'],
                     'n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING')
                continue
            # kwantificering 1 is niet leeg, kwantificering 2 is wel leeg
            elif (row['classificatie_kwantificering1'] != '') & (row['classificatie_kwantificering2'] == ''):
                count_fout_geenkwantificering2 += 1
                kwantificering_waarde = row['classificatie_kwantificering1']
            # kwantificering 1 is wel leeg, kwantificering 2 is niet leeg
            elif (row['classificatie_kwantificering1'] == '') & (row['classificatie_kwantificering2'] != ''):
                count_fout_geenkwantificering1 += 1
                kwantificering_waarde = row['classificatie_kwantificering2']
            elif (row['classificatie_kwantificering1'] != '') & (row['classificatie_kwantificering2'] != ''):
                kwantificering_waarde = row['classificatie_kwantificering1']
            else:
                pass
            
            # In sommige rijen is de kwantificering niet een getal maar tekst. Meestal een getal met een punt erachter '.'
            try:
                int(kwantificering_waarde)
            except(ValueError):
                try:
                    float(kwantificering_waarde)
                except(ValueError):
                    count_fout_kwantificeringistekst += 1
                    print(code_hckar1kar2,row['classificatie_kwantificering1'],'n.v.t. - KWANTIFICERING IS TEKST')
                    list_klasse_indeling.append('n.v.t. - KWANTIFICERING IS TEKST')
                    continue
            
            # Veel leidingen in Vlissingen hebben geen breedte waarde.
            # Pijp breedte wordt uit .ribx gehaald en niet uit .shp!
            # Diameter wordt uit .shp gehaald als reserve!
            try:
                int(row['pijp_breedte'])
            except(ValueError):
                count_fout_bajageenpijpbreedte += 1 
                value_klasse = 'n.v.t. - BAJA GEEN PIJP BREEDTE BESCHIKBAAR VOOR INDELING'
                list_klasse_indeling.append(value_klasse)                                    # Waarde toevoegen
                print(value_klasse)
                continue

            df_streng_breedte = df_streng.loc[df_streng['pijp_breedte'] != ''].reset_index(drop=True)
            
            if int(row['pijp_breedte']) in list_pijp_breedtes:
                round_pijp_breedte = int(row['pijp_breedte'])                      # Oorspronkelijke waarde behouden
            else:
                round_pijp_breedte = round(int(row['pijp_breedte'])/100) * 100     # Afronden naar dichtbijzijnde 100 mm
                
            df_BAJA_selectie = df_tabel_BAJA.loc[
                    df_tabel_BAJA['mm_diameter'] <= round_pijp_breedte].reset_index(drop=True)
            
            df_BAJA_selectie_transposed = df_BAJA_selectie.T
            df_BAJA_selectie_transposed.reset_index(inplace=True)
            
            Mat_nieuw = dict_BAJA_mat[str(row['Mat'])]                      # Dictionary gebruiken om namen om te zetten
            
            if Mat_nieuw == 'Beton':

                # Is 'Mof-/spieverbinding'
                if row['Vrb'] == 'Mof-/spieverbinding':
                    df_baja_value = df_BAJA_selectie_transposed.loc[
                        df_BAJA_selectie_transposed['index'].isin(list_baja_beton_mof)].reset_index(drop=True)
                    df_baja_value['gd_koga'] = df_baja_value['index'].map(dict_BAJA_gd_koga) # Groter dan... toevoegen
                    df_baja_value['klasse'] = df_baja_value['index'].map(dict_BAJA_klasse)   # Klasse toevoegen
                    
                    if float(kwantificering_waarde) == 0:                     # Soms is een waarde 0
                        value_klasse = 1
                    elif float(kwantificering_waarde) > 100:                  # Soms is een waarde > 100
                        value_klasse = 5
                    else:
                        df_baja_value = df_baja_value.loc[
                            ((df_baja_value[0] < float(kwantificering_waarde)) &
                             (df_baja_value['gd_koga'] == 'gd'))].reset_index(drop=True)
                        value_klasse = df_baja_value['klasse'].iloc[-1]                      # Laatste entry gebruiken

                # Is 'Vaar-/moerverbinding'
                elif row['Vrb'] == 'Vaar-/moerverbinding':
                    df_baja_value = df_BAJA_selectie_transposed.loc[
                        df_BAJA_selectie_transposed['index'].isin(list_baja_beton_vaar)].reset_index(drop=True)
                    df_baja_value['gd_koga'] = df_baja_value['index'].map(dict_BAJA_gd_koga) # Groter dan... toevoegen
                    df_baja_value['klasse'] = df_baja_value['index'].map(dict_BAJA_klasse)   # Klasse toevoegen
                    
                    if float(kwantificering_waarde) == 0:                     # Soms is een waarde 0
                        value_klasse = 1
                    elif float(kwantificering_waarde) > 100:                  # Soms is een waarde > 100
                        value_klasse = 5
                    else:
                        df_baja_value = df_baja_value.loc[
                            ((df_baja_value[0] < float(kwantificering_waarde)) &
                             (df_baja_value['gd_koga'] == 'gd'))].reset_index(drop=True)
                        value_klasse = df_baja_value['klasse'].iloc[-1]                      # Laatste entry gebruiken

                # Neem aan dat het 'Mof-/spieverbinding' is
                else:
                    df_baja_value = df_BAJA_selectie_transposed.loc[
                        df_BAJA_selectie_transposed['index'].isin(list_baja_beton_mof)].reset_index(drop=True)
                    df_baja_value['gd_koga'] = df_baja_value['index'].map(dict_BAJA_gd_koga) # Groter dan... toevoegen
                    df_baja_value['klasse'] = df_baja_value['index'].map(dict_BAJA_klasse)   # Klasse toevoegen
                    
                    if float(kwantificering_waarde) == 0:                     # Soms is een waarde 0
                        value_klasse = 1
                    elif float(kwantificering_waarde) > 100:                  # Soms is een waarde > 100
                        value_klasse = 5
                    else:
                        df_baja_value = df_baja_value.loc[
                            ((df_baja_value[0] < float(kwantificering_waarde)) &
                             (df_baja_value['gd_koga'] == 'gd'))].reset_index(drop=True)
                        value_klasse = df_baja_value['klasse'].iloc[-1]                      # Laatste entry gebruiken

            elif Mat_nieuw == 'Kunstof':
                df_baja_value = df_BAJA_selectie_transposed.loc[
                        df_BAJA_selectie_transposed['index'].isin(list_baja_kunstof)].reset_index(drop=True)
                df_baja_value['gd_koga'] = df_baja_value['index'].map(dict_BAJA_gd_koga) # Groter dan... toevoegen
                df_baja_value['klasse'] = df_baja_value['index'].map(dict_BAJA_klasse)   # Klasse toevoegen
                
                if float(kwantificering_waarde) == 0:                     # Soms is een waarde 0
                    value_klasse = 1
                elif float(kwantificering_waarde) > 100:                  # Soms is een waarde > 100
                    value_klasse = 5
                else:
                    df_baja_value = df_baja_value.loc[
                        ((df_baja_value[0] < float(kwantificering_waarde)) &
                        (df_baja_value['gd_koga'] == 'gd'))].reset_index(drop=True)
                    value_klasse = df_baja_value['klasse'].iloc[-1]                      # Laatste entry gebruiken

            elif Mat_nieuw == 'Overig':
                df_baja_value = df_BAJA_selectie_transposed.loc[
                        df_BAJA_selectie_transposed['index'].isin(list_baja_overig)].reset_index(drop=True)
                df_baja_value['gd_koga'] = df_baja_value['index'].map(dict_BAJA_gd_koga) # Groter dan... toevoegen
                df_baja_value['klasse'] = df_baja_value['index'].map(dict_BAJA_klasse)   # Klasse toevoegen
                
                if float(kwantificering_waarde) == 0:                     # Soms is een waarde 0
                    value_klasse = 1
                elif float(kwantificering_waarde) > 100:                  # Soms is een waarde > 100
                    value_klasse = 5                
                else:
                    df_baja_value = df_baja_value.loc[
                        ((df_baja_value[0] < float(kwantificering_waarde)) &
                        (df_baja_value['gd_koga'] == 'gd'))].reset_index(drop=True)
                    value_klasse = df_baja_value['klasse'].iloc[-1]                      # Laatste entry gebruiken
            else:
                count_fout_bajageenmateriaal += 1
                value_klasse = 'n.v.t. - BAJA GEEN MATERIAAL'

            list_klasse_indeling.append(value_klasse)                                    # Waarde toevoegen
            continue
        else:
            pass
        
        # Aanname dat het om een hoekverdraaiing in graden is. 
        if code_hckar1kar2 == 'BAJC':
            
            kwantificering_waarde = -9999                                                 # Reset
            
            # Controle of kwantificering 1 en/of 2 leeg is
            # kwantificering 1 en kwantificering 2 zijn leeg
            if (row['classificatie_kwantificering1'] == '') & (row['classificatie_kwantificering2'] == ''):
                count_fout_geenkwantificering1 += 1
                count_fout_geenkwantificering2 += 1
                list_klasse_indeling.append('n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING')
                print(code_hckar1kar2,row['classificatie_kwantificering1'],row['classificatie_kwantificering2'],
                     'n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING')
                continue
            # kwantificering 1 is niet leeg, kwantificering 2 is wel leeg
            elif (row['classificatie_kwantificering1'] != '') & (row['classificatie_kwantificering2'] == ''):
                count_fout_geenkwantificering2 += 1
                kwantificering_waarde = row['classificatie_kwantificering1']
            # kwantificering 1 is wel leeg, kwantificering 2 is niet leeg
            elif (row['classificatie_kwantificering1'] == '') & (row['classificatie_kwantificering2'] != ''):
                count_fout_geenkwantificering1 += 1
                kwantificering_waarde = row['classificatie_kwantificering2']
            elif (row['classificatie_kwantificering1'] != '') & (row['classificatie_kwantificering2'] != ''):
                kwantificering_waarde = row['classificatie_kwantificering1']
            else:
                pass
            
            # In sommige rijen is de kwantificering niet een getal maar tekst. Meestal een getal met een punt erachter '.'
            try:
                int(kwantificering_waarde)
            except(ValueError):
                try:
                    float(kwantificering_waarde)
                except(ValueError):
                    count_fout_kwantificeringistekst += 1
                    print(code_hckar1kar2,row['classificatie_kwantificering1'],'n.v.t. - KWANTIFICERING IS TEKST')
                    list_klasse_indeling.append('n.v.t. - KWANTIFICERING IS TEKST')
                    continue
            
            df_klasse_selectie = df_code_hc.loc[
                (df_code_hc['Karakterisering 1 code'] == str_new_kar1) &
                (df_code_hc['Karakterisering 2 code'] == str_new_kar2) &
                (df_code_hc['Karakterisering 3 (RibX) code'] == str_new_kar3)].reset_index(drop=True)
            
            df_klasse_selectie.drop(df_klasse_selectie.tail(1).index,inplace=True)
            df_klasse_selectie['Klassegrens: \ngroter dan >'] = pd.to_numeric(
                df_klasse_selectie['Klassegrens: \ngroter dan >'],errors='coerce')
            df_klasse_selectie['Klassegrens: \nkleiner of gelijk aan ≤'] = pd.to_numeric(
                df_klasse_selectie['Klassegrens: \nkleiner of gelijk aan ≤'],errors='coerce')
            
            # Sommige kwantificering waarden zijn 0. 0 kan niet groter dan 0
            if float(kwantificering_waarde) == 0:                     # Soms is een waarde 0
                    value_klasse = 1
            elif float(kwantificering_waarde) > 90:                   # Een waarde boven de 90, wordt 5
                    value_klasse = 5
            else:
                try:
                    df_klasse_indeling = df_klasse_selectie.loc[
                        (df_klasse_selectie['Klassegrens: \ngroter dan >'] <= float(kwantificering_waarde)) &
                        (df_klasse_selectie['Klassegrens: \nkleiner of gelijk aan ≤'] >= float(kwantificering_waarde))
                    ].reset_index(drop=True)
                    value_klasse = df_klasse_indeling['Klasse-indeling'].iloc[0]
                except(IndexError):
                    print('IndexError BAJC')
                    print('hckar1kar2',code_hckar1kar2)
                    print(df_klasse_indeling)
                    print(df_klasse_selectie)
                    print('Kwantificering waarde:',kwantificering_waarde)
                    
                    if (float(kwantificering_waarde) >= 0) & (float(kwantificering_waarde) <= 2):
                        value_klasse = 1
                    elif (float(kwantificering_waarde) > 2) & (float(kwantificering_waarde) <= 5):
                        value_klasse = 3
                    else:
                        value_klasse = 5
                    
            list_klasse_indeling.append(value_klasse)
            continue
        else:
            pass
        
    d = {'code_hckar1kar2': list_code,
         'hc': list_hc,
         'kar1': list_kar1,
         'kar2': list_kar2,
         'kar3': list_kar3,
        'classificatie_kwantificering1': list_classificatie_kwantificering1,
        'klasse_indeling': list_klasse_indeling}
    
    print('len code_hckar1kar2',len(list_code))
    print('len hc',len(list_hc))
    print('len kar1',len(list_kar1))
    print('len kar2', len(list_kar2))
    print('len kar3', len(list_kar3))
    print('len classificatie_kwantificering1',len(list_classificatie_kwantificering1))
    print('len klasse_indeling',len(list_klasse_indeling))
    
    df_inspectiedata_klasse_indeling = pd.DataFrame(d)
    
    df_gemeente_inspectiedata['code_hckar1kar2'] = df_inspectiedata_klasse_indeling['code_hckar1kar2']
    df_gemeente_inspectiedata['classificatie_kwantificering1'] = df_inspectiedata_klasse_indeling['classificatie_kwantificering1']
    df_gemeente_inspectiedata['klasse_indeling'] = df_inspectiedata_klasse_indeling['klasse_indeling']
    
    # Print het aantal fouten
    print("Count Fouten Hoofdcode:",count_fout_hoofdcode)
    print("Count Fouten 'Code niet in bestand':",count_fout_codenietinbestand)
    print("Count Fouten 'Geen kwantificering1':",count_fout_geenkwantificering1)
    print("Count Fouten 'Geen kwantificering2':",count_fout_geenkwantificering2)
    print("Count Fouten 'Kwantificering is tekst':",count_fout_kwantificeringistekst)
    print("Count Fouten 'DataFrame klasseindeling leeg':",count_fout_dataframeklasseindelingleeg)
    print("Count Fouten 'BAJA geen pijp breedte':",count_fout_bajageenpijpbreedte)
    print("Count Fouten 'BAJA geen materiaal':",count_fout_bajageenmateriaal)
    
    

### Functie - Maatstaf Toevoegen
Tekstuele klassen 'Geen Bijzonderheden', 'Waarschuwing' en 'Ingrijpen'.

In [18]:
def func_maatstaf_toevoegen(df_gemeente_inspectiedata):
    # Dictionary iteration in plaats van .iterrows()
    
    print('Maatstaf Toevoegen...')
    
    dict_maatstaf_Waa_ondg = dict(zip(df_maatstaf['Hoofdcode'],df_maatstaf['Waa_ondg']))
    dict_maatstaf_Waa_bovg = dict(zip(df_maatstaf['Hoofdcode'],df_maatstaf['Waa_bovg']))
    dict_maatstaf_Ing_ondg = dict(zip(df_maatstaf['Hoofdcode'],df_maatstaf['Ing_ondg']))
    dict_maatstaf_Ing_bovg = dict(zip(df_maatstaf['Hoofdcode'],df_maatstaf['Ing_bovg']))
    
    list_hc_alleen_ingrijpen = ['BAC','BAO','BAP']        # Geen waarde bij 'waarschuwing'
    list_hc_alleen_waarschuwing = ['BAM','BAN','BDD']     # Bij eerste 3 visuele inspectie nodig voor 'ingrijpen'
    list_hc_alleen_b = ['BBG','BBH']
    list_hc_nvt = ['BCA','BCB','BCC','BCD','BCE','BDA','BDB','BDC','BDE','BDF','BDG',
                  'AEA','AEC','AED','AEF']                # Geen maatstaf classificatie
    
    list_maatstaf = []
    
    dict_gemeente_inspectiedata = df_gemeente_inspectiedata.to_dict('records')
    
    for row in dict_gemeente_inspectiedata:
        
        hoofdcode = row['classificatie_hoofdcode']
        klasse_indeling = row['klasse_indeling']
        str_klasse_indeling = str(row['klasse_indeling'])
        
        if str_klasse_indeling[0:6] == 'n.v.t.':
            list_maatstaf.append('n.v.t.')
        elif hoofdcode in list_hc_alleen_ingrijpen:
            ingrijpen_onder = int(dict_maatstaf_Ing_ondg[hoofdcode])
            ingrijpen_boven = int(dict_maatstaf_Ing_bovg[hoofdcode])
            klasse_indeling = int(klasse_indeling)
            if (klasse_indeling >= ingrijpen_onder) & (klasse_indeling <= ingrijpen_boven):
                list_maatstaf.append('HZ-Ingrijpen')
            else:
                list_maatstaf.append('HZ-Geen Bijzonderheden')
        elif hoofdcode in list_hc_alleen_waarschuwing:
            waarschuwing_onder = int(dict_maatstaf_Waa_ondg[hoofdcode])
            waarschuwing_boven = int(dict_maatstaf_Waa_bovg[hoofdcode])
            klasse_indeling = int(klasse_indeling)
            if (klasse_indeling >= waarschuwing_onder) & (klasse_indeling <= waarschuwing_boven):
                list_maatstaf.append('HZ-Waarschuwing')
            else:
                list_maatstaf.append('HZ-Geen Bijzonderheden')
        elif hoofdcode in list_hc_alleen_b:
            list_maatstaf.append('n.v.t.')
        elif hoofdcode in list_hc_nvt:
            list_maatstaf.append('n.v.t.')
        elif (hoofdcode == 'BAJ') & (klasse_indeling == 'zie tabblad\nBAJ A'):
            list_maatstaf.append('HZ-ANDERS')
        else:
            waarschuwing_onder = int(dict_maatstaf_Waa_ondg[hoofdcode])
            waarschuwing_boven = int(dict_maatstaf_Waa_bovg[hoofdcode])
            ingrijpen_onder = int(dict_maatstaf_Ing_ondg[hoofdcode])
            ingrijpen_boven = int(dict_maatstaf_Ing_bovg[hoofdcode])
            klasse_indeling = int(klasse_indeling)

            if klasse_indeling < waarschuwing_onder:
                list_maatstaf.append('HZ-Geen Bijzonderheden')
            elif (klasse_indeling >= waarschuwing_onder) & (klasse_indeling <= waarschuwing_boven):
                list_maatstaf.append('HZ-Waarschuwing')
            elif (klasse_indeling >= ingrijpen_onder) & (klasse_indeling <= ingrijpen_boven):
                list_maatstaf.append('HZ-Ingrijpen')
            else:
                list_maatstaf.append('HZ-ANDERS')
    
    df_gemeente_inspectiedata['maatstaf'] = list_maatstaf
    print('')                                                                 # Ruimte maken tussen gemeenten
    

### Functie - Data naar per Streng

De data input bij deze functie bestaat uit een geclassificeerde incident/waarneming per rij. In deze functie worden deze data platgeslagen naar per streng (inspectie). Er worden lijsten gemaakt met alle codes, locaties etc. per streng en dus per rij in de nieuwe DataFrame.

streng_uniek heet vanaf nu StrengNam

In [19]:
def func_inspectiedata_perstreng(df_gemeente_leidingen,df_gemeente_inspectiedata):
    
    array_streng = df_gemeente_inspectiedata['streng_uniek'].unique()
    
    col_streng_num = []                        # Unieke code gebasseerd op bestandsnaam en strengnummer
    col_streng_knp_meting_begin = []
    col_streng_knp_meting_eind = []
    col_inspectierichting = []
    col_streng_code = []
    col_streng_klasse = []
    col_streng_maatstaf = []
    col_streng_afstand = []
    col_streng_foto = []
    col_streng_tijd = []
    col_streng_inspectiedatum = []
    
    count_indexerror = 0                       # Aantal IndexError tellen
    list_verwijderen = []                      # Onbruikbare strengen verwijderen bij het maken van DataFrame hieronder
    
    for streng_uniek in array_streng:

        if str(streng_uniek) == 'nan':
            continue
        else:
            pass

        # Gebruik 'streng_uniek'
        df_streng = df_gemeente_inspectiedata.loc[
            df_gemeente_inspectiedata['streng_uniek'] == streng_uniek].reset_index(drop=True)
        df_streng_codes = df_streng.loc[df_streng['classificatie_tijd'] != ''].reset_index(drop=True)
        df_streng_codes['classificatie_code_hckar1kar2'] = (df_streng_codes['classificatie_hoofdcode'] +
                                                       df_streng_codes['classificatie_kar1'] +
                                                       df_streng_codes['classificatie_kar2'])
        df_streng_codes['classificatie_code_hckar1kar2kar3'] = (df_streng_codes['classificatie_hoofdcode'] + '|' +
                                                               df_streng_codes['classificatie_kar1'] + '|' +
                                                               df_streng_codes['classificatie_kar2'] + '|' +
                                                               df_streng_codes['classificatie_kar3'])
        try:
            df_streng_codes['knooppunt_meting_begin'].iloc[0]
        except(IndexError):
            count_indexerror += 1
            list_verwijderen.append(streng_uniek)
            print(df_streng_codes)
            print(streng_uniek)
            continue
        
        col_streng_num.append(streng_uniek)
        col_streng_knp_meting_begin.append(df_streng_codes['knooppunt_meting_begin'].iloc[0])
        col_streng_knp_meting_eind.append(df_streng_codes['knooppunt_meting_eind'].iloc[-1])
        col_inspectierichting.append(df_streng_codes['inspectierichting'].iloc[0])
        col_streng_code.append(df_streng_codes['classificatie_code_hckar1kar2kar3'].to_list())
        col_streng_klasse.append(df_streng_codes['klasse_indeling'].to_list())
        col_streng_maatstaf.append(df_streng_codes['maatstaf'].to_list())
        col_streng_afstand.append(df_streng_codes['classificatie_afstand'].to_list())
        col_streng_foto.append(df_streng_codes['classificatie_foto'].to_list())
        col_streng_tijd.append(df_streng_codes['classificatie_tijd'].to_list())
        col_streng_inspectiedatum.append(df_streng_codes['inspectiedatum'].iloc[0])             # Uitgaan van één datum
        
    d = {'streng_num': col_streng_num,
        'streng_knp_begin': col_streng_knp_meting_begin,
        'streng_knp_eind': col_streng_knp_meting_eind,
        'inspectierichting': col_inspectierichting,
        'list_code': col_streng_code,
        'list_klasse': col_streng_klasse,
        'list_maatstaf': col_streng_maatstaf,
        'list_afstand': col_streng_afstand,
        'list_foto': col_streng_foto,
        'list_tijd': col_streng_tijd,
        'inspectiedatum_ribx': col_streng_inspectiedatum}
    
    df_gemeente_inspectiedata_perstreng = pd.DataFrame(d)
    df_gemeente_inspectiedata_perstreng = df_gemeente_inspectiedata_perstreng[
        ~df_gemeente_inspectiedata_perstreng['streng_num'].isin(list_verwijderen)].reset_index(drop=True)
    
    dict_streng_knp_begin = dict(zip(
        df_gemeente_inspectiedata_perstreng['streng_num'],df_gemeente_inspectiedata_perstreng['streng_knp_begin']))
    dict_streng_knp_eind = dict(zip(
        df_gemeente_inspectiedata_perstreng['streng_num'],df_gemeente_inspectiedata_perstreng['streng_knp_eind']))
    dict_inspectierichting = dict(zip(
        df_gemeente_inspectiedata_perstreng['streng_num'],df_gemeente_inspectiedata_perstreng['inspectierichting']))
    dict_list_code = dict(zip(
        df_gemeente_inspectiedata_perstreng['streng_num'],df_gemeente_inspectiedata_perstreng['list_code']))
    dict_list_klasse = dict(zip(
        df_gemeente_inspectiedata_perstreng['streng_num'],df_gemeente_inspectiedata_perstreng['list_klasse']))
    dict_list_maatstaf = dict(zip(
        df_gemeente_inspectiedata_perstreng['streng_num'],df_gemeente_inspectiedata_perstreng['list_maatstaf']))
    dict_list_afstand = dict(zip(
        df_gemeente_inspectiedata_perstreng['streng_num'],df_gemeente_inspectiedata_perstreng['list_afstand']))
    dict_list_foto = dict(zip(
        df_gemeente_inspectiedata_perstreng['streng_num'],df_gemeente_inspectiedata_perstreng['list_foto']))
    dict_list_tijd = dict(zip(
        df_gemeente_inspectiedata_perstreng['streng_num'],df_gemeente_inspectiedata_perstreng['list_tijd']))
    dict_list_inspectiedatum_ribx = dict(zip(
        df_gemeente_inspectiedata_perstreng['streng_num'],df_gemeente_inspectiedata_perstreng['inspectiedatum_ribx']))
    
    dict_strengnam = dict(zip(df_gemeente_inspectiedata['BHOBJDC000'],df_gemeente_inspectiedata['streng_uniek']))
    df_gemeente_leidingen['StrengNam'] = df_gemeente_leidingen['BHOBJDC000'].map(dict_strengnam)
    
    # Lists toevoegen
    df_gemeente_leidingen['streng_knp_begin'] = df_gemeente_leidingen['StrengNam'].map(dict_streng_knp_begin)
    df_gemeente_leidingen['streng_knp_eind'] = df_gemeente_leidingen['StrengNam'].map(dict_streng_knp_eind)
    df_gemeente_leidingen['inspectierichting'] = df_gemeente_leidingen['StrengNam'].map(dict_inspectierichting)
    df_gemeente_leidingen['list_code'] = df_gemeente_leidingen['StrengNam'].map(dict_list_code)
    df_gemeente_leidingen['list_klasse'] = df_gemeente_leidingen['StrengNam'].map(dict_list_klasse)
    df_gemeente_leidingen['list_maatstaf'] = df_gemeente_leidingen['StrengNam'].map(dict_list_maatstaf)
    df_gemeente_leidingen['list_afstand'] = df_gemeente_leidingen['StrengNam'].map(dict_list_afstand)
    df_gemeente_leidingen['list_foto'] = df_gemeente_leidingen['StrengNam'].map(dict_list_foto)
    df_gemeente_leidingen['list_tijd'] = df_gemeente_leidingen['StrengNam'].map(dict_list_tijd)
    df_gemeente_leidingen['inspectiedatum_ribx'] = df_gemeente_leidingen['StrengNam'].map(dict_list_inspectiedatum_ribx)

### Functie - Counts & Maximum per Klasse
Tellingen en maximum klasse bepalen per leiding. De tellingen vormen de basis van het faalkans proxy model.

Alle hoofdcodes worden één voor één afgegaan. Er wordt een dataframe van de counts gemaakt en een dataframe van het maximum (allebei per code). Vervolgens worden deze twee nieuwe dataframes toegevoegd aan de bestaande dataframe.

In [20]:
def func_counts_en_maximum(df_gemeente_leidingen):
    # Dictionary iteration in plaats van .iterrows()
    
    # Aantal lege rijen printen waar 'list_code' ontbreekt
    print(df_gemeente_leidingen['list_code'].value_counts(dropna=False))
    
    df_gemeente_leidingen = df_gemeente_leidingen.loc[~df_gemeente_leidingen['list_code'].isnull()].reset_index(drop=True)
    
    
    list_hc = ['BAA','BAB','BAC','BAD','BAE','BAF','BAG','BAH','BAI','BAJ','BAK','BAL','BAM','BAN','BAO','BAP',
               'BBA','BBB','BBC','BBD','BBE','BBF','BBH',
               'BCA','BCB','BCC','BCD','BCE',
               'BDA','BDB','BDC','BDD','BDE','BDF','BDG']
    
    # Controle Hoofdcode
    # Nodig omdat gemeente Borsele een float als hoofdcode had
    list_remove = []
    print('--------Start functie TypeError--------')
    for index, row in df_gemeente_leidingen.iterrows():
        
        try:
            for code_total in row['list_code']:
                code_total[0:3]
        except(TypeError):
            list_remove.append(index)
            print('StrengNam:',row['StrengNam'])
            print('code met probleem:     ',code_total)
            print('list code met probleem:',row['list_code'])
            continue
                
    print('list_remove:',list_remove)            
    print(df_gemeente_leidingen['list_code'].value_counts(dropna=False))
    
    df_gemeente_leidingen.drop(list_remove, axis=0, inplace=True)
    dict_iter = df_gemeente_leidingen.to_dict('records')
    
    # Counts
    dict_rijen_code_counts = {}
    index = 0
    for row in dict_iter:
        
        dict_alle_hc = {}

        # Lege cel (geen camera meting)
        if str(row['list_code']) == 'nan':

            for code_hc in list_hc:
                str_col = code_hc + '_count'
                dict_alle_hc[str_col] = np.nan

            dict_rijen_code_counts[index] = dict_alle_hc
            continue
        else:
            pass
        
        
        for code_hc in list_hc:
            count_hc = 0

            for code_total in row['list_code']:
                
                try:
                    if code_total[0:3] == code_hc:
                        count_hc += 1
                    else:
                        pass
                except(TypeError):
                    print('StrengNam:',row['StrengNam'])
                    print('list_code:',row['list_code'])
                    print('code probleem:',code_total)
                    
                    
#                 if code_total[0:3] == code_hc:
#                     count_hc += 1
#                 else:
#                     pass

            str_col = code_hc + '_count'        # Kolomnaam
            dict_alle_hc[str_col] = count_hc    # Count toevoegen

        dict_rijen_code_counts[index] = dict_alle_hc
        index += 1                              # Volgende index

    # Maximum
    dict_rijen_code_mklasse = {}
    index = 0
    for row in dict_iter:
        
        dict_alle_hc = {}

        # Lege cel (geen camera meting)
        if str(row['list_code']) == 'nan':

            for code_hc in list_hc:
                str_col = code_hc + '_max_klasse'
                dict_alle_hc[str_col] = np.nan

            dict_rijen_code_mklasse[index] = dict_alle_hc
            continue
        else:
            pass
        
        # Wel camera meting
        for code_hc in list_hc:
            count_hc = 0

            list_code_klasse = []                           # Lijst met 

            for i in range(0,len(row['list_code'])):

                code_total = row['list_code'][i]
                code_klasse = row['list_klasse'][i]
                if code_total[0:3] == code_hc:
                    list_code_klasse.append(code_klasse)
                else:
                    pass

            if not list_code_klasse:                       # Lijst is leeg
                str_col = code_hc + '_max_klasse'
                dict_alle_hc[str_col] = np.nan
            else:
                check_num = all(isinstance(i_klasse,(int,float)) for i_klasse in list_code_klasse)
                if check_num == True:
                    str_col = code_hc + '_max_klasse'
                    dict_alle_hc[str_col] = max(list_code_klasse)
                else:
                    str_col = code_hc + '_max_klasse'
                    dict_alle_hc[str_col] = np.nan

        dict_rijen_code_mklasse[index] = dict_alle_hc
        index += 1
    
    # Counts en maxima in DataFrames stoppen
    df_cols_hc_counts = pd.DataFrame.from_dict(dict_rijen_code_counts, orient='index')
    df_cols_hc_max = pd.DataFrame.from_dict(dict_rijen_code_mklasse, orient='index')    
    
    # DataFrames samenvoegen
    list_dfs = [df_gemeente_leidingen, df_cols_hc_counts, df_cols_hc_max]
    df_gemeente_leidingen_gis = pd.concat(list_dfs, axis=1)
    
    return df_gemeente_leidingen_gis

### Functie - 3 Klassen
Geen Bijzonderheden, Waarschuwing & Ingrijpen. Deze drie klassen worden door de gemeenten gehanteerd. HZ nog toegevoegd om aan te duiden dat dit extern in bepaald en dat de klanten hopelijk begrijpen dat dit nog een pilot is.

In [21]:
def func_gis_variabelen(list_klasse,list_maatstaf):
    
    if (str(list_klasse) == 'nan') & (str(list_maatstaf) == 'nan'):
        return np.nan, np.nan, np.nan
    else:
        pass
    
    # Klasse
    list_klasse_getal = []
    for klasse in list_klasse:
        str_klasse = str(klasse)
        if str_klasse[0:6] == 'n.v.t.':
            pass
        else:
            int_getal = int(klasse)
            list_klasse_getal.append(int_getal)
            
    # Lijst is leeg, geen getallen
    if not list_klasse_getal:
        return_klasse = 0
    else:
        return_klasse = max(list_klasse_getal)
        
    count_ingrijpen = 0
    count_waarschuwing = 0
        
    # maatstaf
    for maatstaf in list_maatstaf:
        
        if maatstaf == 'HZ-Ingrijpen':
            count_ingrijpen += 1
        elif maatstaf == 'HZ-Waarschuwing':
            count_waarschuwing += 1
        else:
            pass
        
    if count_ingrijpen > 0:
        return_ingrijpen = count_ingrijpen
    else:
        return_ingrijpen = 0
        
    if count_waarschuwing > 0:
        return_waarschuwing = count_waarschuwing
    else:
        return_waarschuwing = 0
        
    return return_klasse, return_ingrijpen, return_waarschuwing



In [22]:
df_faalkans

,Hoofdcode,Karakterisering 1 code,Toestandsaspect,Faalwijze,faalkans_klasse1_basis,faalkans_klasse2_1binnenxm,faalkans_klasse2_meerdere,faalkans_klasse3_1binnenxm,faalkans_klasse3_meerdere,faalkans_klasse4_1binnenxm,faalkans_klasse4_meerdere,faalkans_klasse5_1binnenxm,faalkans_klasse5_meerdere
0,BAA,NaN,Deformatie,Stabiliteit/Instorting,1,2,3,3,4,4,5,5,6
1,BAB,NaN,Scheur,Lekkage,2,2,3,3,4,4,5,5,6
2,BAC,NaN,Breuk/Instorting,Stabiliteit/Instorting,1,5,5,5,5,6,6,6,6
3,BAD,NaN,Defecte bakstenen of defect metselwerk,Stabiliteit/Instorting,2,2,2,3,4,3,4,4,5
4,BAE,NaN,Ontbrekende metselspecie,Stabiliteit/Instorting,2,2,2,3,4,3,4,4,5
5,BAF,NaN,Oppervlakteschade,Stabiliteit/Instorting,2,2,2,3,4,3,4,4,5
6,BAG,NaN,Instekende inlaat,Lekkage,1,2,3,3,4,4,5,5,6
7,BAH,NaN,Defecte aansluiting,Lekkage,2,2,3,3,4,4,5,5,6
8,BAI,A,Indringend afdichtingsmateriaal,Afstroom,1,2,3,3,3,3,4,4,5
9,BAI,Z,indringend\nafdichtingsmateriaal\nandere afdic...,Afstroom,1,2,3,3,3,3,4,4,5


# Faalkans Proxy
Er is niet een 'echte' faalkans (als in een 1 op zoveel kans dat...). We maken gebruik van een proxy op basis van het aantal incidenten in een leiding.

In [23]:
def func_faalkans(df_gemeente_leidingen_temp,afstand_intensiteit,df_faalkans):
    # Dictionary iteration in plaats van .iterrows()
    
    dict_iter = df_gemeente_leidingen_temp.to_dict('records')
    
    list_hc = ['BAA','BAB','BAC','BAD','BAE','BAF','BAG','BAH','BAI','BAJ','BAK','BAL','BAM','BAN','BAO','BAP',
               'BBA','BBB','BBC','BBD','BBE','BBF','BDD']
    
    dict_col_intensiteit = {1: 'faalkans_klasse1_basis',
                            2: 'faalkans_klasse2_1binnenxm',
                            3: 'faalkans_klasse3_1binnenxm',
                            4: 'faalkans_klasse4_1binnenxm',
                            5: 'faalkans_klasse5_1binnenxm'}
    
    dict_col_frequentie = {1: 'faalkans_klasse1_basis',
                           2: 'faalkans_klasse2_meerdere',
                           3: 'faalkans_klasse3_meerdere',
                           4: 'faalkans_klasse4_meerdere',
                           5: 'faalkans_klasse5_meerdere'}
    
    dict_rijen_code_faalkans = {}
    index = 0
    
    for row in dict_iter:
                                             # Index nodig om DFs samen te voegen
        dict_alle_hc = {}
        
        for hc in list_hc:
            
            # Lege cel (geen camera meting)
            if str(row['StrengNam']) == 'nan':

                for hc in list_hc:
                    str_col = hc + '_kans'
                    dict_alle_hc[str_col] = np.nan

                dict_rijen_code_faalkans[index] = dict_alle_hc
                continue
            else:
                pass
            
            str_col = hc + '_kans'                      # Naam kolom
            str_count = hc + '_count'                   # Om de count kolommen te selecteren
            str_max = hc + '_max_klasse'                # Om de max kolommen te selecteren
            
            hc_count = row[str_count]
            hc_max = row[str_max]
            
            if np.isnan(hc_max):
                hc_max = 1
            else:
                pass
            
            # Basis faalkans
            hc_faalkans_basis = df_faalkans['faalkans_klasse1_basis'].loc[
                df_faalkans['Hoofdcode'] == hc].iloc[0]
            hc_faalkans_frequentie = 0                                                                 # Reset
            hc_faalkans_intensiteit = 0                                                                # Reset
            
            # Intensiteit berekening
            if hc_count == 0:
                hc_intensiteit = 21
            elif (np.isnan(row['LenLei'])) | (str(row['LenLei']) == 'nan'):
                print('Geen Lengte Leiding')
                hc_intensiteit = 20
            else:
                hc_intensiteit = row['LenLei'] / hc_count
            
            
            if (hc_count > 1) & (hc_max >= 2):
                hc_faalkans_frequentie = df_faalkans[dict_col_frequentie[hc_max]].loc[
                    df_faalkans['Hoofdcode'] == hc].iloc[0]
            else:
                pass
            
            # hc_intensiteit is kleiner, dus gem. meer incidenten per 20 m
            if (hc_intensiteit <= afstand_intensiteit) & (hc_intensiteit != 0):
                hc_faalkans_intensiteit = df_faalkans[dict_col_intensiteit[hc_max]].loc[
                    df_faalkans['Hoofdcode'] == hc].iloc[0]
            else:
                pass
                
            list_faalkansen = [hc_faalkans_basis,hc_faalkans_frequentie,hc_faalkans_intensiteit]
            hc_faalkans_max = max(list_faalkansen)
            
            dict_alle_hc[str_col] = hc_faalkans_max
            
        dict_rijen_code_faalkans[index] = dict_alle_hc
        index += 1                                                # Volgende index
        
    df_cols_hc_faalkans = pd.DataFrame.from_dict(dict_rijen_code_faalkans, orient='index')
    
    list_dfs = [df_gemeente_leidingen_temp, df_cols_hc_faalkans]
    df_gemeente_leidingen_gis = pd.concat(list_dfs, axis=1)
    
    return df_gemeente_leidingen_gis

### Functie - Coördinaten voor GIS
Coördinaten voor GIS moeten in een bepaalde format zijn om ingelezen te kunnen worden

In [24]:
def func_coordinates_gis(list_coordinates_rd):
    
    # LineString, niet LineStringZ
    # MultiLineString heeft twee haken nodig
    str_start = 'LineString ('
    str_end = ')'
    
    str_list_coordinates_rd = str(list_coordinates_rd)
    
    str_stripped = str_list_coordinates_rd.replace('[','')
    str_stripped = str_stripped.replace(']','')
    str_stripped = str_stripped.replace('(','')
    str_stripped = str_stripped.replace(')','')
    str_stripped = str_stripped.replace(',','')
    
    # Komma tussen coördinaten
    str_comma = re.sub(r'(\s\S*?)\s', r'\1, ',str_stripped)
    
    str_new = str_start + str_comma + str_end
    
    return str_new

# Machine Learning Preparatie

### Functie - ML Preparatie voor Regressie
Deze functie selecteert de kolommen die nuttig zijn voor een regressie. De targets zijn in dit geval 'count_ingrijpen' en 'count_waarschuwing'. Dus de hoeveelheid incidenten per leiding die deze klasses kennen.

In [25]:
def func_ml_preparatie_reg(df_export_qgis_zeeland):
    
    list_cols_ml_reg = ['wkt_geom','BHOBJDC000','LeiNam','KnpNamB','KnpNamE','StrGeb',
                        'StrNam','coordinates_type','coordinates_rd','Gemeente',
                        'knooppunt_rd_begin','knooppunt_rd_eind','StrengNam','list_code',
                        'list_klasse','list_maatstaf','list_afstand','list_foto','list_tijd','inspectiedatum_ribx',
                        'Mat','Vrb','Br','Hgt','LenLei','Vrm','BobB','BobE','TypFun','TypLei',
                        'BegDat_datum','DatAanpGep_datum','DatInsp_datum',
                        'Stab','Afs','WatDicht',
                        'alle_codes_max_klasse','count_ingrijpen','count_waarschuwing']
    
    df_ml_reg = df_export_qgis_zeeland[list_cols_ml_reg].dropna(subset=['StrengNam']).reset_index(drop=True)
    
    return df_ml_reg


Deze functie selecteert de kolommen die gebruikt worden voor een classificatie model die kijkt hoe de gemeenten de leidingen indelen. In principe is de ML model de keuzemethodiek van de gemeenten.

In [26]:
def func_ml_preparatie_clf(df_export_qgis_zeeland):
    
    list_cols_ml_clf = ['wkt_geom','BHOBJDC000','LeiNam','KnpNamB','KnpNamE','StrGeb',
                        'StrNam','Mat','Vrb','coordinates_type','coordinates_rd',
                        'knooppunt_rd_begin','knooppunt_rd_eind','StrengNam','list_code',
                        'list_klasse','list_maatstaf','list_afstand','list_foto','list_tijd','inspectiedatum_ribx',
                        'BAA_count', 'BAB_count', 'BAC_count', 'BAD_count', 'BAE_count', 'BAF_count', 'BAG_count',
                        'BAH_count', 'BAI_count', 'BAJ_count', 'BAK_count', 'BAL_count',
                        'BAM_count', 'BAN_count', 'BAO_count', 'BAP_count', 'BBA_count',
                        'BBB_count', 'BBC_count', 'BBD_count', 'BBE_count', 'BBF_count',
                        'BBH_count', 'BCA_count', 'BCB_count', 'BCC_count', 'BCD_count',
                        'BCE_count', 'BDA_count', 'BDB_count', 'BDC_count', 'BDD_count',
                        'BDE_count', 'BDF_count', 'BDG_count', 'BAA_max_klasse',
                        'BAB_max_klasse', 'BAC_max_klasse', 'BAD_max_klasse', 'BAE_max_klasse',
                        'BAF_max_klasse', 'BAG_max_klasse', 'BAH_max_klasse', 'BAI_max_klasse',
                        'BAJ_max_klasse', 'BAK_max_klasse', 'BAL_max_klasse', 'BAM_max_klasse',
                        'BAN_max_klasse', 'BAO_max_klasse', 'BAP_max_klasse', 'BBA_max_klasse',
                        'BBB_max_klasse', 'BBC_max_klasse', 'BBD_max_klasse', 'BBE_max_klasse',
                        'BBF_max_klasse', 'BBH_max_klasse', 'BCA_max_klasse', 'BCB_max_klasse',
                        'BCC_max_klasse', 'BCD_max_klasse', 'BCE_max_klasse', 'BDA_max_klasse',
                        'BDB_max_klasse', 'BDC_max_klasse', 'BDD_max_klasse', 'BDE_max_klasse',
                        'BDF_max_klasse', 'BDG_max_klasse',
                        'alle_codes_max_klasse',
                        'Stab','Afs','WatDicht']
    
    df_ml_clf = df_export_qgis_zeeland[list_cols_ml_clf].dropna(subset=['StrengNam']).reset_index(drop=True)
    
    return df_ml_clf

In [27]:
df_faalkans

,Hoofdcode,Karakterisering 1 code,Toestandsaspect,Faalwijze,faalkans_klasse1_basis,faalkans_klasse2_1binnenxm,faalkans_klasse2_meerdere,faalkans_klasse3_1binnenxm,faalkans_klasse3_meerdere,faalkans_klasse4_1binnenxm,faalkans_klasse4_meerdere,faalkans_klasse5_1binnenxm,faalkans_klasse5_meerdere
0,BAA,NaN,Deformatie,Stabiliteit/Instorting,1,2,3,3,4,4,5,5,6
1,BAB,NaN,Scheur,Lekkage,2,2,3,3,4,4,5,5,6
2,BAC,NaN,Breuk/Instorting,Stabiliteit/Instorting,1,5,5,5,5,6,6,6,6
3,BAD,NaN,Defecte bakstenen of defect metselwerk,Stabiliteit/Instorting,2,2,2,3,4,3,4,4,5
4,BAE,NaN,Ontbrekende metselspecie,Stabiliteit/Instorting,2,2,2,3,4,3,4,4,5
5,BAF,NaN,Oppervlakteschade,Stabiliteit/Instorting,2,2,2,3,4,3,4,4,5
6,BAG,NaN,Instekende inlaat,Lekkage,1,2,3,3,4,4,5,5,6
7,BAH,NaN,Defecte aansluiting,Lekkage,2,2,3,3,4,4,5,5,6
8,BAI,A,Indringend afdichtingsmateriaal,Afstroom,1,2,3,3,3,3,4,4,5
9,BAI,Z,indringend\nafdichtingsmateriaal\nandere afdic...,Afstroom,1,2,3,3,3,3,4,4,5


# Main - .ribx Processing
- Runtime Vlissingen: ongeveer 2 uur
- Runtime Sluis: 1 uur 10 minuten (dict it. 1 uur)
- Runtime Veere: 12 minuten
- Runtime Reimerswaal: 25 minuten (dict it. 21)

Lijst van gemeenten moet aangepast worden op basis van de beschikbare data. Directory van de .ribx en .shp bestanden moeten gedefinieerd worden.

Vlissingen: Twee StrengNam die 2 keer voorkomen. Rest uniek. N88_11K8_1478_007, N88_11K8_1443_008, 6135 rijen data

Veere: allemaal unieke StrengNam

In [28]:
def func_main():
    
#     directory_ribx = r'C:\Users\haze0029\OneDrive - HZ University of Applied Sciences\Documents\Projects\Project_DataScience\RIOBASE\Data\Data_Beeld\Ribx_Bestanden'
#     directory_shp = r'C:\Users\haze0029\OneDrive - HZ University of Applied Sciences\Documents\Projects\Project_DataScience\RIOBASE\Data\GIS'
#     directory_exports = r'C:\Users\haze0029\OneDrive - HZ University of Applied Sciences\Documents\Projects\Project_DataScience\RIOBASE\Exports\Ribx_naar_GIS'
#     directory_ML = r'C:\Users\haze0029\OneDrive - HZ University of Applied Sciences\Documents\Projects\Project_DataScience\RIOBASE\Data\ML'
    
    
    directory_gemeenten = r'C:\Users\haze0029\OneDrive - HZ University of Applied Sciences\Documents\Projects\Project_DataScience\RIOBASE\RioBaseOplevering\Gemeenten'
    
#     os.chdir(directory_ribx)
    
    list_dfs_gemeenten = []
    
    afstand_intensiteit = 20                                                        # Parameter nodig voor faalkans proxy
    
    # Gemeenten in Zeeland
    list_gemeenten = ['Goes','Hulst','Kapelle','Middelburg','Noord-Beveland',
                      'Reimerswaal','Schouwen-Duiveland','Sluis',
                      'Terneuzen','Tholen','Veere','Vlissingen']
    list_gemeenten = ['Borsele','Hulst','Kapelle','Middelburg',
                      'Reimerswaal','Sluis','Tholen',
                      'Veere','Vlissingen']
    list_gemeenten = ['Borsele']
    
    for file_foldernaam in os.listdir(directory_gemeenten):
        
        if file_foldernaam in list_gemeenten:                                       # Is het een valide gemeente
            gemeente_naam = file_foldernaam                                         # Naam van gemeente
            
            # Nieuwe directory voor .ribx
            directory_gemeente_ribx = directory_gemeenten + '\\' + gemeente_naam + '\Inspectie\Direct_van_gemeente'
            
            # Nieuwe directory voor .shp
            directory_gemeente_shp_portaal = directory_gemeenten + '\\' + gemeente_naam + '\Riool_Portaal'
            directory_gemeente_shp_direct = directory_gemeenten + '\\' + gemeente_naam + '\Riool_Direct'
            
            # Nieuwe directory voor exports
            directory_gemeente_exports = directory_gemeenten + '\\' + gemeente_naam + '\Inspectie'
            
        else:
            continue
        
        
        # .shp (shape file) data van gemeente ophalen en inlezen
        naam_shp = '0'                                                                # Reset
        
        if len(os.listdir(directory_gemeente_shp_portaal)) == 0:
            os.chdir(directory_gemeente_shp_direct)
            print('Geen data van portaal')
            for file_naam in os.listdir(directory_gemeente_shp_direct):
                if (file_naam[-4:] == '.shp') & (file_naam[13:22] == 'LEIDINGEN'):
                    
                    naam_shp = file_naam
                    break
                else:
                    pass
        else:
            os.chdir(directory_gemeente_shp_portaal)
            for file_naam in os.listdir(directory_gemeente_shp_portaal):
                print('Data van portaal')
                if (file_naam[-4:] == '.shp') & (file_naam[13:22] == 'LEIDINGEN'):
                    naam_shp = file_naam
                    break
                else:
                    pass
        
#         # .shp (shape file) data van gemeente ophalen en inlezen
#         os.chdir(directory_gemeente_shp_portaal)                                      # Directory veranderen
#         naam_shp = '0'                                                                # Reset
#         for file_naam in os.listdir(directory_gemeente_shp_portaal):                  # Loop om juiste bestand te vinden
            
#             if (file_naam[-4:] == '.shp') & (file_naam[13:22] == 'LEIDINGEN'):
#                 naam_shp = file_naam
#                 break
#             else:
#                 pass
#         naam_shp = gemeente_naam + '_lei.shp'
        
        gemeente_leidingen_shapefile = shapefile.Reader(naam_shp, encoding = 'latin1')
        df_gemeente_leidingen = func_shapefile_lei_inladen(gemeente_naam,gemeente_leidingen_shapefile)  # functie
        
        df_gemeente_leidingen['knooppunt_rd_begin'],\
        df_gemeente_leidingen['knooppunt_rd_eind'] = zip(*df_gemeente_leidingen.apply(
            lambda row: func_coordinates_rd_split(row['coordinates_rd']),axis=1))
        
        # .ribx inspectiedata inladen en verwerken
        df_gemeente_inspectiedata = func_ribx_processing(directory_gemeente_ribx)   # Inspectiedata uit .ribx bestanden
        df_gemeente_inspectiedata_prekoppeling = df_gemeente_inspectiedata.copy()   # Versie opslaan voor controle
        
        df_gemeente_inspectiedata, df_gemeente_inspectiedata_mm = func_koppeling_ribx_shp(df_gemeente_leidingen,df_gemeente_inspectiedata)  # Koppeling
        df_gemeente_inspectiedata_postkoppeling = df_gemeente_inspectiedata.copy()  # Versie opslaan voor controle
        
        df_gemeente_inspectiedata = func_materiaal_vervangen(df_gemeente_leidingen,df_gemeente_inspectiedata)
        
        # De strengen (leidingen) een schadeklasse toekennen
        func_classificatie_strengen(df_gemeente_inspectiedata,
                               dict_BAJA_mat, dict_BAJA_klasse, dict_BAJA_gd_koga,
                               list_baja_beton_mof, list_baja_beton_vaar, list_baja_kunstof, list_baja_overig,
                               list_pijp_breedtes)
        
        func_maatstaf_toevoegen(df_gemeente_inspectiedata)
        
        func_inspectiedata_perstreng(df_gemeente_leidingen,df_gemeente_inspectiedata)
        
        # Targets voor machine learning voorbereiden, ook nodig voor risicobeeld
        df_gemeente_leidingen_gis = func_counts_en_maximum(df_gemeente_leidingen)
        
        # 
        df_gemeente_leidingen_gis = func_faalkans(df_gemeente_leidingen_gis,afstand_intensiteit,df_faalkans)
        
        # 
        df_gemeente_leidingen_gis['alle_codes_max_klasse'],\
        df_gemeente_leidingen_gis['count_ingrijpen'],\
        df_gemeente_leidingen_gis['count_waarschuwing'] = zip(*df_gemeente_leidingen_gis.apply(
            lambda row: func_gis_variabelen(row['list_klasse'],row['list_maatstaf']),axis=1))
        
        df_gemeente_leidingen_gis['wkt_geom'] = df_gemeente_leidingen_gis.apply(
            lambda row: func_coordinates_gis(row['coordinates_rd']),axis=1)
        
        list_cols = df_gemeente_leidingen_gis.columns.to_list()
        list_cols.insert(0, list_cols.pop(list_cols.index('wkt_geom')))             # move the column to head of list
        df_gemeente_leidingen_gis = df_gemeente_leidingen_gis.loc[:, list_cols]     # use ix to reorder
        
        df_export_qgis = df_gemeente_leidingen_gis.dropna(subset=['StrengNam']).reset_index(drop=True)
        
        # Individuele exports
        os.chdir(directory_gemeente_exports)
        
        str_gemeente_excel = 'Ribx_GIS_Preparatie_Gemeente_' + gemeente_naam + '.xlsx'
        str_gemeente_csv = 'Ribx_GIS_Preparatie_Gemeente_' + gemeente_naam + '.csv'
        
        df_export_qgis.to_excel(str_gemeente_excel)
        df_export_qgis.to_csv(str_gemeente_csv,sep=';')
        
        list_dfs_gemeenten.append(df_export_qgis)              # DataFrame van gemeente toevoegen aan list
        
    
    # DataFrame met alle gemeenten
#     os.chdir(directory_exports)
    
#     df_export_qgis_zeeland = pd.concat(list_dfs_gemeenten)
#     df_export_qgis_zeeland.to_excel('Ribx_GIS_Preparatie_Zeeland.xlsx')
#     df_export_qgis_zeeland.to_csv('Ribx_GIS_Preparatie_Zeeland.csv',sep=';')
    
#     os.chdir(directory_ML)
#     df_ml_reg = func_ml_preparatie_reg(df_export_qgis_zeeland)
#     df_ml_clf = func_ml_preparatie_clf(df_export_qgis_zeeland)
    
#     df_ml_reg.to_csv('Ribx_ML_Preparatie_Regressie.csv',sep=';')
#     df_ml_clf.to_csv('Ribx_ML_Preparatie_Classificatie.csv',sep=';')
    
    return df_export_qgis, df_gemeente_inspectiedata_prekoppeling, df_gemeente_inspectiedata_postkoppeling, df_gemeente_inspectiedata_mm
    

In [29]:
%%time
df_export_qgis, df_gemeente_inspectiedata_prekoppeling, df_gemeente_inspectiedata_postkoppeling, df_gemeente_inspectiedata_mm = func_main()
df_export_qgis

Data van portaal
Data van portaal
Data van portaal
Data van portaal
Data van portaal
Data van portaal
Data van portaal
Data van portaal
Data van portaal
Data van portaal
Data van portaal
Data van portaal
Data van portaal
Data van portaal
Data van portaal
Data van portaal
Data van portaal
Shape Files Leidingen Inladen... Borsele
LineString    4766
Name: coordinates_type, dtype: int64
0    Vuilwaterriool
1    Vuilwaterriool
2    Vuilwaterriool
3    Vuilwaterriool
4    Vuilwaterriool
5    Vuilwaterriool
6    Vuilwaterriool
7    Vuilwaterriool
8     Gemengd riool
9    Vuilwaterriool
Name: TypLei, dtype: object
---BHOBJDC000 Ontbreekt---
---BHOBJDC000 Aangemaakt--- met GROEP
Oorspronkelijke datum: 1999-12-31
Nieuwe datum: 1999-12-31 00:00:00
Lengtes voor conversie 0    80.68
1    71.64
2    48.46
3    52.25
4    67.78
5    71.74
6    72.59
7    49.41
8    31.66
9    50.77
Name: LenLei, dtype: float64
Aantal cijfers voor komma LenLei: 2
Aantal cijfers voor komma Br:     3
Aantal cijfers voor

BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BBBA   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BBBA   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BBBC   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BAJA   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BBBA   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BBBC   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BAJA   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BAJA   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BBBA   n.v.t. - GEEN KWANTIFICERING 1 OF

BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BAJA   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCH

BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 

BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERIN

...

BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHI

BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 

BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHI

BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BBBB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BBBB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDDB   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BBBA   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BBBA   n.v.t. - GEEN KWANTIFICERING 1 OF 2 BESCHIKBAAR VOOR INDELING
BDD   n.v.t. - GEEN KWANTIFICERING 1 O

,wkt_geom,BHOBJDC000,LeiNam,TypLei,KnpNamB,KnpNamE,BobB,BobE,LenLei,Diam,...,BBA_kans,BBB_kans,BBC_kans,BBD_kans,BBE_kans,BBF_kans,BDD_kans,alle_codes_max_klasse,count_ingrijpen,count_waarschuwing
0,"LineString (46399.93 388459.44, 46449.97 38839...",LEI_08739_08737,1824.0,Vuilwaterriool,08739,08737,-1.14,-1.46,80.68,250.0,...,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0
1,"LineString (46337.55 388421.53, 46281.31 38837...",LEI_08741_08742,1825.0,Vuilwaterriool,08741,08742,-2.08,-1.70,71.64,250.0,...,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,0.0,1.0
2,"LineString (46337.55 388421.53, 46367.62 38838...",LEI_08741_08754,1826.0,Vuilwaterriool,08741,08754,-2.56,-2.51,48.46,315.0,...,2.0,2.0,3.0,1.0,1.0,2.0,1.0,2.0,0.0,3.0
3,"LineString (46281.31 388377.16, 46239.42 38834...",LEI_08742_08743,1827.0,Vuilwaterriool,08742,08743,-1.70,-1.40,52.25,200.0,...,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0
4,"LineString (46239.4 388430.43, 46281.31 388377...",LEI_08747_08742,1828.0,Vuilwaterriool,08747,08742,-1.15,-1.56,67.78,250.0,...,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3515,"LineString (40094.83 382718.97, 40094.18 38271...",LEI_021026_021027,4703.0,Hemelwaterriool,021026,021027,-0.55,-0.55,4.04,250.0,...,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0
3516,"LineString (40094.83 382718.97, 40109.15 38279...",LEI_021026_021025,4704.0,Hemelwaterriool,021026,021025,0.00,0.25,79.99,250.0,...,2.0,2.0,2.0,1.0,1.0,2.0,1.0,5.0,1.0,2.0
3517,"LineString (40113.42 382819.46, 40109.15 38279...",LEI_021025X_021025,4705.0,Hemelwaterriool,021025X,021025,0.70,0.25,22.20,160.0,...,2.0,2.0,2.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0
3518,"LineString (50944.23 384620.95, 50940.85 38461...",LEI_101013X_101013,4708.0,Hemelwaterriool,101013X,101013,0.00,0.00,4.95,400.0,...,2.0,2.0,2.0,1.0,1.0,2.0,1.0,5.0,11.0,6.0


In [30]:
# os.chdir(r'C:\Downloads')
# df_gemeente_inspectiedata_prekoppeling.to_excel('gemeente_inspectiedata_prekoppeling.xlsx')
# df_gemeente_inspectiedata_postkoppeling.to_excel('gemeente_inspectiedata_postkoppeling.xlsx')

# df_gemeente_inspectiedata_prekoppeling.head()

In [67]:
df_test = df_gemeente_inspectiedata_mm[
    df_gemeente_inspectiedata_mm['BHOBJDC000'] == 'LEI_08259_08260'].copy()

In [68]:
df_test['NamRibx'].value_counts()

Inspecties Borsele 23-9-22 mod    21
Name: NamRibx, dtype: int64

In [69]:
df_test.columns

Index(['streng', 'AAB', 'AAC', 'knooppunt_meting_begin', 'knooppunt_rd_begin',
       'knooppunt_meting_eind', 'knooppunt_rd_eind', 'opdrachtgever',
       'soort_locatie', 'grondeigendom', 'naam_rioolstelsel', 'wijk', 'norm',
       'ABQ', 'vorm', 'pijp_hoogte', 'pijp_breedte', 'materiaal',
       'verharding_wegdek', 'verbinding_type', 'nap_bob_beginpunt',
       'nap_bob_eindpunt', 'ACJ', 'ACK', 'leidingcurve', 'inspectiemethode',
       'inspectiedatum', 'ABH', 'inspectierichting', 'ZC',
       'classificatie_hoofdcode', 'classificatie_kar1', 'classificatie_kar2',
       'classificatie_kwantificering1', 'classificatie_kwantificering2',
       'classificatie_text', 'classificiate_plaats_omtrek1',
       'classificatie_plaats_omtrek2', 'classificatie_afstand',
       'classificatie_meelopende_schade', 'ZC-K', 'ZC-L', 'classificatie_foto',
       'classificatie_tijd', 'classificatie_kar3', 'ZC-P',
       'algemene_opmerking', 'straat', 'woonplaats', 'AEC', 'AED', 'AEA',
       'NamRib

In [70]:
df_test['inspectiedatum']

55907    2013-06-11
55908    2013-06-11
55909    2013-06-11
55910    2013-06-11
55911    2013-06-11
55912    2013-06-11
55913    2013-06-11
55914    2013-06-11
55915    2013-06-11
55916    2013-06-11
55917    2013-06-11
55918    2013-06-11
55919    2013-06-11
55920    2013-06-11
55921    2013-06-11
55922    2013-06-11
55923    2013-06-11
55924    2013-06-11
55925    2013-06-11
55926    2013-06-11
55927    2013-06-11
Name: inspectiedatum, dtype: object

In [71]:
df_test[['inspectiedatum','classificatie_hoofdcode']][0:60]

,inspectiedatum,classificatie_hoofdcode
55907,2013-06-11,BCD
55908,2013-06-11,BDD
55909,2013-06-11,BDD
55910,2013-06-11,BAF
55911,2013-06-11,BAF
55912,2013-06-11,BCA
55913,2013-06-11,BAI
55914,2013-06-11,BAI
55915,2013-06-11,BCA
55916,2013-06-11,BCA


In [47]:
df_gemeente_inspectiedata_mm['straat'].unique()

array([nan, 'nieuweweg', 'slotstraat', 'florishoek',
       'prinses wilhelminastraat', 'delische weg', 'zuidelijke achterweg',
       'burgemeester vogelaarstr', 'zuidhoekstraat', 'kolewiegenhoek',
       'noordelijke achterweg', 'torenstraat', 'schansstraat',
       'hellenburgstraat', 'molenpad', 'quistenburg', 'troyeweg',
       'kalootstraat', 'borszeestraat', 'diepeneestraat',
       'wielingenstraat', 'zuidsingel', 'monsterweg', 'westsingel',
       'zuidstraat', 'wolphaartsweg', 'plein', 'oostsingel',
       'cleeuwendamme', 'everingestraat', 'noordstraat', 'noordsingel',
       'weelweg', 'draaiweitje', 'ooststraat', 'weststraat',
       'catalijneweg', 'platteweg', 'maaiklinkstraat',
       'm a slaakwegstraat', 'korteweg', 'coudorpseweg',
       'van tilburghstraat', 'smitsweg', 'paulushoekseweg',
       'schoondijksedijk', 'grindweg', 'commejansstraat',
       'hellewoudstraat', 'langeviele', 'zomerstraat', 'van hattumstraat',
       'markt', 'breestraat', 'dijkstraat', 'tr

In [32]:
df_export_qgis[df_export_qgis['BHOBJDC000'] == 'LEI_15349_15352']

,wkt_geom,BHOBJDC000,LeiNam,TypLei,KnpNamB,KnpNamE,BobB,BobE,LenLei,Diam,...,BBA_kans,BBB_kans,BBC_kans,BBD_kans,BBE_kans,BBF_kans,BDD_kans,alle_codes_max_klasse,count_ingrijpen,count_waarschuwing
983,"LineString (45648.57 383851.81, 45639.76 38385...",LEI_15349_15352,4048.0,Gemengd riool,15349,15352,-2.53,-2.53,10.49,500.0,...,2.0,2.0,2.0,2.0,1.0,5.0,1.0,5.0,0.0,7.0


In [33]:
df_gemeente_inspectiedata_postkoppeling.head()

,streng,AAB,AAC,knooppunt_meting_begin,knooppunt_rd_begin,knooppunt_meting_eind,knooppunt_rd_eind,opdrachtgever,soort_locatie,grondeigendom,...,algemene_opmerking,straat,woonplaats,AEC,AED,AEA,NamRibx,streng_uniek,BHOBJDC000,LeiNam
0,08246-08117-1,08246,"[45786.28, 388825.76]",08246,"[45786.28, 388825.76]",08117,"[44626.83, 389378.87]",Gemeente Borsele Internet,Z,C,...,NaN,NaN,NaN,NaN,NaN,NaN,Inspecties Borsele 23-9-22 mod,Inspecties Borsele 23-9-22 mod_08246-08117-1,LEI_08246_08117,4171.0
1,08246-08117-1,08246,"[45786.28, 388825.76]",08246,"[45786.28, 388825.76]",08117,"[44626.83, 389378.87]",Gemeente Borsele Internet,Z,C,...,NaN,NaN,NaN,NaN,NaN,NaN,Inspecties Borsele 23-9-22 mod,Inspecties Borsele 23-9-22 mod_08246-08117-1,LEI_08246_08117,4171.0
2,08246-08117-1,08246,"[45786.28, 388825.76]",08246,"[45786.28, 388825.76]",08117,"[44626.83, 389378.87]",Gemeente Borsele Internet,Z,C,...,NaN,NaN,NaN,NaN,NaN,NaN,Inspecties Borsele 23-9-22 mod,Inspecties Borsele 23-9-22 mod_08246-08117-1,LEI_08246_08117,4171.0
3,08246-08117-1,08246,"[45786.28, 388825.76]",08246,"[45786.28, 388825.76]",08117,"[44626.83, 389378.87]",Gemeente Borsele Internet,Z,C,...,NaN,NaN,NaN,NaN,NaN,NaN,Inspecties Borsele 23-9-22 mod,Inspecties Borsele 23-9-22 mod_08246-08117-1,LEI_08246_08117,4171.0
4,08246-08117-1,08246,"[45786.28, 388825.76]",08246,"[45786.28, 388825.76]",08117,"[44626.83, 389378.87]",Gemeente Borsele Internet,Z,C,...,NaN,NaN,NaN,NaN,NaN,NaN,Inspecties Borsele 23-9-22 mod,Inspecties Borsele 23-9-22 mod_08246-08117-1,LEI_08246_08117,4171.0


In [34]:
d = {'Hoofdcode': ['BDD','BDD','BDD','BDD','BDD'],
    'Beschrijving': ['Waterpeil','Waterpeil','Waterpeil','Waterpeil','Waterpeil'],
    'Klasse-indeling': [1,2,3,4,5],
    'Klassegrens: \ngroter dan >': [0,10,25,50,75],
    'Klassegrens: \nkleiner of gelijk aan ≤': [10,25,50,75,100],
    'code_hckar1kar2': ['BDD','BDD','BDD','BDD','BDD']}

df_klasse_selectie = pd.DataFrame(d)

In [35]:
df_klasse_indeling = df_klasse_selectie.loc[
                (df_klasse_selectie['Klassegrens: \ngroter dan >'] <= 25) &
                (df_klasse_selectie['Klassegrens: \nkleiner of gelijk aan ≤'] >= 25)
                ].reset_index(drop=True)

In [36]:
df_klasse_selectie

,Hoofdcode,Beschrijving,Klasse-indeling,Klassegrens: \ngroter dan >,Klassegrens: \nkleiner of gelijk aan ≤,code_hckar1kar2
0,BDD,Waterpeil,1,0,10,BDD
1,BDD,Waterpeil,2,10,25,BDD
2,BDD,Waterpeil,3,25,50,BDD
3,BDD,Waterpeil,4,50,75,BDD
4,BDD,Waterpeil,5,75,100,BDD


In [37]:
df_klasse_indeling

,Hoofdcode,Beschrijving,Klasse-indeling,Klassegrens: \ngroter dan >,Klassegrens: \nkleiner of gelijk aan ≤,code_hckar1kar2
0,BDD,Waterpeil,2,10,25,BDD
1,BDD,Waterpeil,3,25,50,BDD


In [38]:
df_klasse_indeling['Klasse-indeling'].iloc[0]

2